In [1]:
from datetime import datetime
import rasterio , os , glob  , geojson
from pathlib import PureWindowsPath
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
import shutil
from collections import defaultdict

import rioxarray
from rasterio.mask import mask
import rasterio.plot as rplt
from rasterio.features import rasterize
from rasterio.plot import show
from rasterio.features import shapes
from rasterio import plot
from rasterstats import zonal_stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import ScalarFormatter
from shapely.geometry import LineString, Point
import math

from matplotlib.colors import Normalize
import matplotlib.cm as cm
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.text as mtext
from collections import defaultdict
from matplotlib import colormaps

from h3 import h3
import h3pandas
import contextily as cx
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import geopandas as gpd
from IPython.display import Image , display
import warnings
warnings.filterwarnings('ignore')
# plt.rcParams['figure.figsize']=9,4
import xarray as xr
from shapely.geometry import mapping, Polygon
import datetime as dt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.patches as mpatches

from dateutil.relativedelta import relativedelta
from rasterio.warp import calculate_default_transform, reproject, Resampling
from geopandas.tools import overlay
from pyproj import CRS
from os.path import exists
import time
from pathlib import Path
import fiona
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import shapefile
from shapely.geometry import shape, Point
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
# Stats heavy-lifting
from libpysal.weights import KNN
from sklearn.preprocessing import robust_scale
from sklearn.cluster import AgglomerativeClustering
import osmnx as ox
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, TransformedBbox, BboxPatch, BboxConnector)

from collections import defaultdict
from itertools import combinations

width=8
height=7


In [2]:
# socio_econmic=False
# cckp=False
socio_econmic=True
cckp=True

# base_dir= '/Users/ipshitakarmakar/Dropbox (MIT)/WB/Future city scan' # Project directory
base_dir= 'C:/Users/Aziz/Dropbox/CRP/FCS' # Project directory
# output= f'{base_dir}/output' # csv and figure output directory
# data= f'{base_dir}/data'  # Input data directory
# shp = f'{data}/test_cities/Buffered_Chittagong_Final.shp' # City shapefile
# shapefiles = f'{data}/test_cities' # City shapefiles output dir
country = 'BGD'
city= 'Chittagong'
#Mapping 
WGS84=4326
EPSG_str= 'EPSG:4326'
hexbin_res=8

def set_paths(base_dir):
    data = os.path.join(base_dir, 'data') 
    output = os.path.join(base_dir, 'output') 
    shapefiles = os.path.join(output, 'shapefiles') 
    maps = os.path.join(output, 'maps') 
    rasters = os.path.join(output, 'rasters') 
    tables = os.path.join(output, 'tables') 
    
    dirs_list= [data, output, base_dir, shapefiles , maps , rasters , tables]
    for dir in dirs_list:
        if not os.path.exists(dir):
            os.mkdir(dir)
    return data, shapefiles , maps , rasters , output ,tables

data, shapefiles , maps , rasters , output ,tables = set_paths(base_dir)

# [8:54 PM] Ipshita Karmakar
# variablename_ssp_year

City and supra and subcity admin shapefiles

In [3]:
shp = f'{shapefiles}/Buffered_Chittagong_Final.shp' # City shapefile
country_shp = f'{shapefiles}/bgd_admbnda_adm0_bbs_20201113.shp' # City shapefile
lowest_admin_shp = f'{shapefiles}/bgd_admbnda_adm4_bbs_20201113.shp' # City shapefile
second_lowest_admin_shp = f'{shapefiles}/bgd_admbnda_adm3_bbs_20201113.shp' # City shapefile
vector_file = gpd.read_file(shp).to_crs(WGS84)
country_vector_file = gpd.read_file(country_shp).to_crs(WGS84)
lowest_admin_vector_file = gpd.read_file(lowest_admin_shp).to_crs(WGS84)
second_lowest_admin_vector_file = gpd.read_file(second_lowest_admin_shp).to_crs(WGS84)


In [4]:
def enumerate_items(source):
    print("/n")
    for ele in enumerate(source): 
        print(ele)

def list_df_columns(df):
    field_list = list(df)
    enumerate_items(field_list)
    return field_list

def percentage_formatter(x, pos):
    return f'{x * 100 :,.0f}'

def millions_formatter(x, pos):
    return f'{x / 1000000 :,.0f}'


def hundred_thousand_formatter(x, pos):
    return f'{x / 100000 :,.0f}'

def billions_formatter(x, pos):
    return f'{x / 1000000000 :,.0f}'

# For loading shapefiles into geopandas dataframe
def load_shapefile(shp):
    data = gpd.read_file(shp).to_crs(WGS84)
    return data

# For loading shapefiles as a geopandas dataframe and buffering it.
def load_shapefile_buffer(shp):
    data = gpd.read_file(shp).to_crs(WGS84).buffer(0.016) #remember to reproject =2 kilomers
    return data

# For re-projecting input vector layer to raster projection 
def reproject_gpdf(input_gpdf, raster):
    proj = raster.crs.to_proj4()
    reproj = input_gpdf.to_crs(proj)
    return reproj

# clipping the raster to the buffered AOI and exporting it. 
def clip_and_export_raster_fathom(input_gpdf, raster , out_raster):
    Vector=input_gpdf
    with rasterio.open(raster) as src:
        Vector=Vector.to_crs(src.crs)
        # print(Vector.crs)
        out_image, out_transform=mask(src,Vector.geometry,crop=True)
        no_data_value=0
        out_image = np.where(out_image<0,no_data_value,out_image) #replaced negative values with 
        # out_image = np.where(((out_image<0) & (out_image>99)),no_data_value,out_image) #replaced negative values with 
        # out_image = np.where(out_image>99,no_data_value,out_image) #replaced negative values with 
        out_meta=src.meta.copy() # copy the metadata of the source DEM
        
    out_meta.update({
        "driver":"Gtiff",
        "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })
                
    with rasterio.open(out_raster,'w',**out_meta) as dst:
        dst.write(out_image)


# clipping the raster to the buffered AOI and exporting it. 
def clip_and_export_raster(input_gpdf, raster , out_raster):
    Vector=input_gpdf
    with rasterio.open(raster) as src:
        Vector=Vector.to_crs(src.crs)
        # print(Vector.crs)
        out_image, out_transform=mask(src,Vector.geometry,crop=True)
        no_data_value=0
        out_image = np.where(out_image<0,no_data_value,out_image) #replaced negative values with 
        out_meta=src.meta.copy() # copy the metadata of the source DEM
        
    out_meta.update({
        "driver":"Gtiff",
        "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })
                
    with rasterio.open(out_raster,'w',**out_meta) as dst:
        dst.write(out_image)



def reproject_rasters(crs , unprojected_raster, projected_raster):
    if not exists(projected_raster):
        with rasterio.open(unprojected_raster) as src:
            dst_crs = 'EPSG:' + str(crs)
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(projected_raster, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)





# For selecting which raster statistics to calculate
def list_statistics(stat):
    out_stats = stat
    return out_stats

# For selecting which raster statistics to calculate
def list_statistics_urbanland():
    out_stats = ['mean']
    return out_stats

# For calculating zonal statistics
def calculate_zonal_stats(vector, raster, stats):
    # Run zonal statistics, store result in geopandas dataframe
    result = zonal_stats(vector, raster, stats=stats, geojson_out=True)
    geostats = gpd.GeoDataFrame.from_features(result)
    for c in stats:
        geostats[c] = np.round(geostats[c] , decimals= 2)
    return geostats

# out_image = np.where(out_image<0,no_data_value,out_image) #replaced negative values with 


# For generating raster from zonal statistics result.
def export_zonal_stats_raster(zdf, raster, stat, out_raster, no_data='y'):
    meta = raster.meta.copy()
    out_shape = raster.shape
    transform = raster.transform
    dtype = raster.dtypes[0]
    field_list = list_df_columns(stats)
    index = 0 #first statistic for now
    zone = zdf[field_list[index]]
    stats = list_statistics(stat)
    shapes = ((geom,value) for geom, value in zip(zdf.geometry, zone))
    burned = rasterize(shapes=shapes, fill=0, out_shape=out_shape, transform=transform)
    # show(burned)
    meta.update(dtype=rasterio.float32, nodata=0)
    # Optional to set nodata values to min of stat
    if no_data == 'y':
        # cutoff = min(zone.values)
        cutoff < 0
        # print("Setting nodata cutoff to: ", cutoff)
        burned[burned < cutoff] = 0 
    with rasterio.open(out_raster, 'w', **meta) as out:
        no_data_value=0
        out.write_band(1, burned)
    # print("Zonal Statistics Raster generated")

def visualize_popdynamics_figure(df_for_heatmap, country, city, subdir):
    # Create a mask for the NaN values
    mask = df_for_heatmap.isnull()
    # Create a heatmap using seaborn
    fig, ax = plt.subplots(figsize=(14,5))
    stats = list_statistics(stat)
    for c in stats:
        if c=="sum":
            df_formatted =df_for_heatmap.applymap( lambda val: f'{val / 100000:,.0f}')
            fmt = ''
        else:
            fmt = '.0f'
        sns.heatmap(df_for_heatmap,  annot=df_formatted, cmap='YlGnBu', cbar=False, fmt=fmt,
                    square=True, annot_kws={'size': 10},
                    linewidths=1, linecolor='gray',
                    cbar_kws={'label': 'Value'}, ax=ax)

        ax.patch.set_edgecolor('black')  
        ax.patch.set_linewidth(1)  
        plt.title(f"Projected Population(100k) \n {city}")
        plt.savefig(f"{maps}/{country}_{city}_{subdir}_{c}.jpeg" , dpi=300, bbox_inches='tight')
        plt.show()

def visualize_popdynamics_graph(df_for_graph, country, city, subdir):
    df_for_graph['Year'] = pd.to_datetime(
                            df_for_graph['Year'],
                            format='%Y'
                            )
    # plot data
    g = sns.lineplot(data=df_for_graph, x="Year", y="value", 
                    hue="Scenario"  , linewidth=2.5                     
                     )
     
    g.set(xlabel='Year', ylabel='Population (100k)')
    # g.axes.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, p: f'{y/1000000:.0f}'))
    g.axes.yaxis.set_major_formatter(ticker.FuncFormatter(hundred_thousand_formatter))
    # Tell matplotlib to interpret the x-axis values as dates
    g.xaxis_date()
    g.get_figure().savefig(f"{maps}/{country}_{city}_{subdir}_graph.jpeg" , dpi=300, bbox_inches='tight')

def visualize_gdp_figure(df_for_heatmap, country, city, subdir):
    # Create a mask for the NaN values
    mask = df_for_heatmap.isnull()
    # Create a heatmap using seaborn
    fig, ax = plt.subplots(figsize=(14,5))
    stats = list_statistics(stat)
    for c in stats:
        if c=="sum":
            df_formatted =df_for_heatmap.applymap( lambda val: f'{val / 1000000000:,.0f}' )
            fmt = ''
        else:
            fmt = '.0f'
        sns.heatmap(df_for_heatmap,  annot=df_formatted, cmap='YlGnBu', cbar=False, fmt=fmt,
                    square=True, annot_kws={'size': 10},
                    linewidths=1, linecolor='gray',
                    cbar_kws={'label': 'Value'}, ax=ax)

        ax.patch.set_edgecolor('black')  
        ax.patch.set_linewidth(1)  
        plt.title(f"Projected GDP of {city}\n in Billions USD")
        plt.savefig(f"{maps}/{country}_{city}_{subdir}_heatmap.jpeg" , dpi=300, bbox_inches='tight')
        plt.show()

def visualize_gdp_graph(df_for_graph, country, city, subdir):
    df_for_graph['Year'] = pd.to_datetime(
                            df_for_graph['Year'],
                            format='%Y'
                            )
    # plot data
    g = sns.lineplot(data=df_for_graph, x="Year", y="value", 
                     hue="Scenario"  , linewidth=2.5
                     )
    g.set(xlabel='Year', ylabel='GDP (Billions USD)')
    g.axes.yaxis.set_major_formatter(ticker.FuncFormatter(billions_formatter))
    # Tell matplotlib to interpret the x-axis values as dates
    g.xaxis_date()
    g.get_figure().savefig(f"{maps}/{country}_{city}_{subdir}_graph.jpeg" , dpi=300, bbox_inches='tight')

def visualize_urbanland_figure(df_for_heatmap, country, city, subdir):
    # Create a mask for the NaN values
    mask = df_for_heatmap.isnull()
    # Create a heatmap using seaborn
    fig, ax = plt.subplots(figsize=(14,5))
    df_formatted =df_for_heatmap.applymap( lambda val: f'{val*100 :,.0f}%')
    fmt = ''
    sns.heatmap(df_for_heatmap,  annot=df_formatted, cmap='YlGnBu', cbar=False, fmt=fmt,
                square=True, annot_kws={'size': 10},
                linewidths=1, linecolor='gray',
                cbar_kws={'label': 'Value'}, ax=ax)

    ax.patch.set_edgecolor('black')  
    ax.patch.set_linewidth(1)  
    plt.title(f"Projected % Urban Land of \n {city} ")
    plt.savefig(f"{maps}/{country}_{city}_{subdir}_heatmap.jpeg" , dpi=300, bbox_inches='tight')
    plt.show()

def visualize_urbanland_graph(df_for_graph, country, city, subdir):
    df_for_graph['Year'] = pd.to_datetime(
                            df_for_graph['Year'],
                            format='%Y'
                            )
    # plot data
    g = sns.lineplot(data=df_for_graph, x="Year", y="value", 
                    hue="Scenario"  , linewidth=2.5                     
                    )
    g.set(xlabel='Year', ylabel='% Urban Land')
    g.axes.yaxis.set_major_formatter(ticker.FuncFormatter(percentage_formatter))
    # Tell matplotlib to interpret the x-axis values as dates
    g.xaxis_date()
    g.get_figure().savefig(f"{maps}/{country}_{city}_{subdir}_graph.jpeg" , dpi=300, bbox_inches='tight')


def visualize_image(dir):
    for file in os.listdir(dir):
        file_path=os.path.join(dir, file)
        print(file_path)
        display(Image(filename=file_path))

def overlay_sub_city_tier(shapefiles, shp, shp1, crs):
    city = gpd.read_file(shp).to_crs(crs)
    sub_city = gpd.read_file(shp1).to_crs(crs)
    ax = city.to_crs(crs).plot(figsize=(8, 8) , \
        facecolor='none', edgecolor='black'
        )
    sub_city.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='red',alpha=0.2)
    # sub_city.plot()
    # return sub_city

def create_hexbins(shapefiles, shp, hexbin_res, WGS84):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    poly = vector_file.geometry.unary_union
    gdf_boundary = gpd.GeoDataFrame(geometry=[poly],crs=vector_file.crs)
    # gdf_h3 = gdf_boundary.h3.polyfill(9,explode=True)
    gdf_h3 = gdf_boundary.h3.polyfill(hexbin_res,explode=True)
    gdf_h3 = gdf_h3.set_index('h3_polyfill').h3.h3_to_geo_boundary()
    output_shapefile= f'{shapefiles}/h3_grid.shp'
    gdf_h3.to_file(output_shapefile)
    hexbins_projected = gpd.read_file(output_shapefile).to_crs(WGS84)
    # gdf_h3.plot()
    return hexbins_projected


def hexbin_zonal_stats(shp, hexbins_projected , crs, raster , out_raster, statistic):
    input_gpdf= gpd.read_file(shp).to_crs(crs)
    Vector=input_gpdf
    with rasterio.open(raster) as src:
        Vector=Vector.to_crs(src.crs)
        # print(f'Gdf srs: {Vector.crs}')
        out_image, out_transform=mask(src,Vector.geometry,crop=True)
        no_data_value=0
        out_image = np.where(out_image<0,no_data_value,out_image) #replaced negative values with 0
        out_meta=src.meta.copy() # copy the metadata of the source DEM
        
    out_meta.update({
        "driver":"Gtiff",
        "height":out_image.shape[1], # height starts with shape[1]
        "width":out_image.shape[2], # width starts with shape[2]
        "transform":out_transform
    })
                
    with rasterio.open(out_raster,'w',**out_meta) as dst:
        dst.write(out_image)

    # add zonal stats to the hexbin gpd dataframe and replace nans with 0 
    hexbins_projected[statistic] = zonal_stats(hexbins_projected, out_raster ,stats=statistic)
    hexbins_projected[statistic] = [item[statistic] for item in hexbins_projected[statistic]]
    hexbins_projected[statistic].fillna(0,inplace=True)
    max=int(hexbins_projected.loc[hexbins_projected[statistic].idxmax()][statistic])
    min=int(hexbins_projected.loc[hexbins_projected[statistic].idxmin()][statistic])
    
    return hexbins_projected , max , min

# Change colormap by truncating
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

# ,vmin=0,  vmax=int(max) , scheme='quantiles'
def export_map_categorical(vector_file,  hexagons, legend_title,legends_format, crs ,cmap,  visualize_column, title, map_output):
    fig = plt.figure(figsize=(width, height))
    ax = fig.add_subplot(111)
    ax = hexagons.to_crs(crs).plot(ax=ax, 
                                   column= visualize_column, \
                                   linewidth=0 , \
                                   alpha=0.6, categorical=True, cmap=cmap, \
                                   legend=True,  # Add legend 
                                   legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1.01, 1.01), 
                                                'fmt':legends_format,
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'x-small',
                                                # 'fontweight': 'bold'
                                                # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                }  
                                        )

    vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
    cx.add_basemap(ax, 
                source=cx.providers.Esri.WorldImagery,  
                crs=crs,
                attribution=False, #No citation
                ) 
    # plt.title(f'{title}',fontsize=16)
    plt.xticks([])
    plt.yticks([])
        
    leg1 = ax.get_legend()
    # Set markers to square shape
    for ea in leg1.legendHandles:
        ea.set_marker('s')
    leg1.set_title(f'{legend_title}')
    # ax.title.set_text(f'{title}') 
    ax.set_title(title,  fontweight='bold', wrap=True)
    ax.axis('off')
    plt.tight_layout()
    ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)
    
def export_map(map_dir, vector_file,  hexagons, legend_title,legends_format, crs ,cmap,  visualize_column, title, map_output):
    fig = plt.figure(figsize=(width, height))
    ax = fig.add_subplot(111)
    ax = hexagons.to_crs(crs).plot(ax=ax, 
                                   column= visualize_column, \
                                   linewidth=0 , \
                                   alpha=0.6, scheme="quantiles", cmap=cmap, \
                                   legend=True,  # Add legend 
                                   legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1.01, 1.01), 
                                                'fmt':legends_format,
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'x-small',
                                                # 'fontweight': 'bold'
                                                # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                }  
                                        )

    vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
    cx.add_basemap(ax, 
                source=cx.providers.Esri.WorldImagery,  
                crs=crs,
                attribution=False, #No citation
                ) 
    # plt.title(f'{title}',fontsize=16)
    plt.xticks([])
    plt.yticks([])
        
    leg1 = ax.get_legend()
    # Set markers to square shape
    for ea in leg1.legendHandles:
        ea.set_marker('s')
    leg1.set_title(f'{legend_title}')
    # ax.title.set_text(f'{title}')
    ax.set_title(title, fontsize=10, fontweight='bold', wrap=True)
    ax.axis('off')
    plt.tight_layout()
    ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)
    
def export_map_osm(map_dir, vector_file,  hexagons, legend_title, legends_format , crs ,cmap,  visualize_column, title, map_output):
    fig = plt.figure(figsize=(width, height))
    ax = fig.add_subplot(111)
    ax = hexagons.to_crs(crs).plot(ax=ax, \
                                   column= visualize_column, 
                                   linewidth=0 , 
                                   alpha=0.6, scheme="quantiles", cmap=cmap, 
                                   legend=True,  # Add legend \
                                   legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1.01, 1.01), 
                                                'fmt':legends_format,
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'x-small',
                                                # 'fontweight': 'bold'
                                                # title_fontsizeint or {'xx-small', 'x-small', 'small'
                                                }  
                                        )
    
    vector_file.to_crs(crs).plot(ax=ax,facecolor='none', edgecolor='k',alpha=0.1)
    # cx.add_basemap(ax, source=cx.providers.Stamen.TonerLite,  crs=crs , zoom=12) 
    cx.add_basemap(ax,
                    zoom="auto",
                    # zoom=10,
                    crs=vector_file.crs.to_string(),
                    # source=cx.providers.CartoDB.Voyager,
                    source=cx.providers.OpenStreetMap.Mapnik,
                    attribution=False, #No citation
                    # attribution_size=1
                    )
    plt.xticks([])
    plt.yticks([])
    leg1 = ax.get_legend()
    # Set markers to square shape
    for ea in leg1.legendHandles:
        ea.set_marker('s')
    leg1.set_title(f'{legend_title}')
    ax.set_title(title,  fontweight='bold', wrap=True)
    ax.axis('off')
    plt.tight_layout()
    ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)
    
def export_map_rasters(map_dir, shp, input_raster, crs , cmap, title, max_for_sublots, Year, ssp):

    # dir_list = os.listdir(input_rasters_path)
    # print("Files and directories in '", input_rasters_path, "' :") 
    # # print the list
    # print(dir_list)

    # Read raster
    b1 = rasterio.open(input_raster)
    b2 = rasterio.open(input_raster)
    b3 = rasterio.open(input_raster)
    b4 = rasterio.open(input_raster)

    # Read geometry
    shapefile = gpd.read_file(shp).to_crs(crs) 
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(6,6), constrained_layout=True) #
    p1  = rasterio.plot.show(b1, interpolation='bilinear',  ax=ax[0,0], cmap=cmap, title=f'Year:{Year} SSP:{ssp}')
    p2  = rasterio.plot.show(b2, interpolation='bilinear',  ax=ax[0,1], cmap=cmap, title=f'Year:{Year} SSP:{ssp}')
    p3  = rasterio.plot.show(b3, interpolation='bilinear',  ax=ax[1,0], cmap=cmap, title=f'Year:{Year} SSP:{ssp}')
    p4  = rasterio.plot.show(b4, interpolation='bilinear',  ax=ax[1,1], cmap=cmap, title=f'Year:{Year} SSP:{ssp}')

    shapefile.plot(ax=p1,  facecolor='none', edgecolor='black',alpha=0.2 )
    shapefile.plot(ax=p2,  facecolor='none', edgecolor='black',alpha=0.2)
    shapefile.plot(ax=p3,  facecolor='none', edgecolor='black',alpha=0.2)
    shapefile.plot(ax=p4,  facecolor='none', edgecolor='black',alpha=0.2 ,  lw=0.7)
    
    normalizer=Normalize(vmin=0,vmax=max_for_sublots)
    im=cm.ScalarMappable(cmap=cmap,norm=normalizer)
    # remove the x and y ticks
    plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
    fig.suptitle(title, fontsize=15)
    # plt.tight_layout()
    fig.colorbar(im, ax=ax.ravel().tolist(), location='bottom', shrink=0.7)
    plt.show()
    # ax.figure.savefig(f"{map_dir}/{city}_{visualize_column}_truncated_cmap_max_div_4_2.png")

def export_figure_with_heading( document_output, mappath, title):  
    document = Document(document_output)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(title)
    r.add_picture(mappath)
    r.add_text('Notes here')
    document.save(document_output)
    # return print('doc saved')

def polygonize(raster_file ,crs, output_shapefile,   driver, mask_value, raster_val, ssp, Year):
    # mask = None
    with rasterio.Env():
        with rasterio.open(raster_file) as src:
            image = src.read(1) # first band
            no_data_value=0
            image = np.where(image<0,no_data_value,image) #replaced negative values with 0
            results = (
            {'properties': {raster_val : v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=mask_value, transform=src.transform)))

    # The result is a generator of GeoJSON features
    geoms = list(results)
    # first feature
    # print(geoms[0])
    gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
    gpd_polygonized_raster['Scenario'] = ssp
    gpd_polygonized_raster['Year'] = Year
    crs_utm = CRS.from_user_input(crs)
    driver= 'ESRI Shapefile'
    gpd_polygonized_raster.to_file(output_shapefile, driver, crs=crs_utm)
    return gpd_polygonized_raster


def polygonize_with_negatives(raster_file ,crs, output_shapefile,   driver, mask_value, raster_val, ssp, Year):
    # mask = None
    with rasterio.Env():
        with rasterio.open(raster_file) as src:
            image = src.read(1) # first band
            # no_data_value=0
            # image = np.where(image<0,no_data_value,image) #replaced negative values with 0
            results = (
            {'properties': {raster_val : v}, 'geometry': s}
            for i, (s, v) 
            in enumerate(
                shapes(image, mask=mask_value, transform=src.transform)))

    # The result is a generator of GeoJSON features
    geoms = list(results)
    # first feature
    # print(geoms[0])
    gpd_polygonized_raster  = gpd.GeoDataFrame.from_features(geoms)
    gpd_polygonized_raster['Scenario'] = ssp
    gpd_polygonized_raster['Year'] = Year
    crs_utm = CRS.from_user_input(crs)
    driver= 'ESRI Shapefile'
    gpd_polygonized_raster.to_file(output_shapefile, driver, crs=crs_utm)
    return gpd_polygonized_raster



def clip_raster_shapefiles(shp, output_shapefile,crs,output_geojson, output_csv, clipped_output_shapefile):
    city_shapefile=   gpd.read_file(shp).to_crs(crs)
    # city_shapefile.plot(aspect=1)
    raster_shapefile=   gpd.read_file(output_shapefile).to_crs(crs)
    # raster_shapefile.plot(aspect=1)
    newdf = overlay(raster_shapefile, city_shapefile, how="intersection")
    # newdf.plot(aspect=1)
    driver= 'ESRI Shapefile'
    newdf.to_file(clipped_output_shapefile, driver)
    driver='GeoJSON'
    newdf.to_file(output_geojson, driver=driver)
    newdf["centroid"] = newdf["geometry"].centroid
    newdf['lon'] = newdf['centroid'].x
    newdf['lat'] = newdf['centroid'].y
    newdf["center"] = newdf.dissolve().centroid #center
    newdf['center_lon'] = newdf['center'].x
    newdf['center_lat'] = newdf['center'].y
    newdf = newdf.drop(columns=['geometry', 'center', 'centroid'])
    newdf.to_csv(output_csv)
    return newdf


def merge_JsonFiles(dir, section):
    folder = Path(dir)
    shapefiles = folder.glob(f"polygonized_clipped_{section}*.shp")
    gdf = pd.concat([gpd.read_file(shp) for shp in shapefiles]).pipe(gpd.GeoDataFrame)
    output_geojson= f'{dir}/panel_{section}.geojson'
    driver='GeoJSON'
    gdf.to_file(output_geojson, driver=driver)

    # json_pattern = os.path.join(dir, f'geojson_{section}*.geojson')
    # file_list = glob.glob(json_pattern)
    # collection = []
    # for file in file_list:
    #     with open(file) as f:
    #         layer = geojson.load(f)
    #         collection.append(layer)
    #         print(f"{layer} appended")
    # geo_collection = geojson.GeometryCollection(collection)
    # # geo_collection = collection #geojson.GeometryCollection(collection)
    # with open(f'{dir}/panel_{section}.geojson', 'w') as f:
    #     geojson.dump(geo_collection, f)
    #     print(f"dumped {dir}")
   

def export_cckp_data(section):
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    appended_data = []
    for subdir in section:
        combined_results = []
        extension = '.nc'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    variable_name = file_name.split("_")[0] #.upper()
                    Year = file_name.split("_")[-1].upper()
                    ssp = file_name.split("-")[-3].upper() 
                    # ssp = subdir.upper()  #Already created 
                    # print(file_name,  file_name_path, subdir, Year, ssp, variable_name)
                    # dataset = xr.open_dataset(file_name_path, decode_coords="all")
                    dataset = xr.open_dataset(file_name_path)
                    clean_var_name= variable_name.replace('-', '_')
                    clean_var_name= clean_var_name.replace(':', '')
                    dataset=dataset.rename(name_dict={f'{variable_name}':f'{clean_var_name}'})
                    var_long_label= dataset.variables[clean_var_name].attrs['long_name'].capitalize()
                    var_long_label= var_long_label.replace(':', '')
                    # print(variable_name ,"----", clean_var_name, "----", var_long_label)
                    # start_date = "2040-01-01"
                    # end_date = "2040-12-31"
                    gpdf = gpd.read_file(shp) #
                    gpdf['centroid'] = gpdf['geometry'].centroid
                    gpdf['lat'] = gpdf['centroid'].y
                    gpdf['lon'] = gpdf['centroid'].x
                    lat= gpdf['centroid'].y
                    lon= gpdf['centroid'].x
                    # df = dataset[clean_var_name].sel(time=slice(start_date, end_date), lat=slice(int(lat) , int(lat)),lon=slice(int(lon) , int(lon)))
                    df = dataset[clean_var_name].sel( lat=slice(int(lat) , int(lat)),lon=slice(int(lon) , int(lon)))
                    df=df.to_dataframe( )
                    df=df.dropna().reset_index()
                    df[subdir]=df[clean_var_name]
                    df["Variable"]=var_long_label 
                    df["SSP"]=ssp
                    try:
                        df_date = df[df[[subdir]].apply(pd.to_datetime, errors='coerce').isna().all(axis=1)]
                        df_date[subdir] = df_date[subdir] / pd.to_timedelta(1, unit='D')
                        appended_data.append(df_date)
                    except:
                        pass
                        # print(f"Passing {var_long_label}")
                        appended_data.append(df)
        appended_data = pd.concat(appended_data, axis=0, join='inner').sort_index()
        # appended_data = pd.concat(appended_data, axis=0).sort_index()
        appended_data.to_csv(f"{tables}/{country}_{city}_{subdir}_panel.csv")
    return appended_data

def export_cckp_graph(df , x_var_name, y_var_names, subdir, city):
    df["date"] = pd.to_datetime(df[f"{x_var_name}"]) #.month
    # df['month_year'] = df['date_column'].dt.to_period('M')
    # y_var_names=df[y_var_names].unique()
    for ssp in df["SSP"].unique():
        for y_var in df[y_var_names].unique():
            sub_df = df[(df["SSP"]==ssp) & (df[y_var_names]==y_var)]
            sub_df=sub_df.sort_values([y_var_names, "SSP"], ascending=[True, True])
            # Set a Style
            sns.set_style('whitegrid')
            # Create plot
            ax = sns.lineplot(data=sub_df, x=x_var_name, 
                            y=subdir , errorbar=None, 
                            hue=y_var_names, style=y_var_names, markers=True
                            )
            # Set Title and Labels
            ax.set_title(f'{subdir} in {city} for {ssp}', 
            fontdict={'size': 13, 'weight': 'bold'})
            ax.set_xlabel(f'Month')
            # ax.set_ylabel(f'{y_var}')
            # Adjust the Legend
            plt.legend(frameon=False, loc='lower left', bbox_to_anchor=(0, -.3), ncol=1)
            # Format the date axis to be prettier.
            # ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
            # ax.xaxis.set_minor_locator(mdates.DayLocator())
            ax.xaxis.set_major_locator(mdates.AutoDateLocator(interval_multiples=False))
            # # Increase size and change color of axes ticks
            GREY91 = "#e8e8e8"
            ax.tick_params(axis="x", length=12, color=GREY91)
            ax.tick_params(axis="y", length=8, color=GREY91)
            # Remove the Spines
            sns.despine()
            plt.show()
            ax.figure.savefig(f"{maps}/{country}_{city}_{subdir}_graph.png",  bbox_inches='tight', dpi=300) #, transparent=True  

    return df #print(f"{subdir} graph exported {df}")

def get_lat_lon(s):
    # split by comma
    l = s[1:-1].split(',')
    # get lat lon without direction
    lat = int(l[0]) + ( float(l[1][:-1]) / 60) # -1 for skipping direction letter
    lon = int(l[2]) + ( float(l[3][:-1]) / 60)
    # check cardinal direction letter
    if l[1][-1] in ['S', 's']: lat = -lat        
    if l[3][-1] in ['W', 'w']: lon = -lon 
    return lat, lon

# s = "(38,40.1365N, 75,4.23933W)"
# # lat, lon = get_lat_lon(s)
# # lat, lon

def get_city_poolygon(shp):
    # get the shapes
    r = shapefile.Reader(shp)
    shapes = r.shapes()
    # build a shapely polygon from your shape
    polygon = shape(shapes[0])   
    # read your shapefile
    return polygon

def check(lon, lat):
    # build a shapely point from your geopoint
    point = Point(lon, lat)
    # the contains function does exactly what you want
    return polygon.contains(point)

def create_storm_csvs(data, section):
    for subdir in section:      
        file_list = glob.glob(os.path.join( os.path.join(data, subdir), "STORM_DATA_*.txt"))
        corpus = []
        files = []
        for file_path in file_list:
            # file = open(file_path, 'r')
            # for line in file.readlines():
            #     fname = line.rstrip().split(',') #using rstrip to remove the \n
            #     corpus.append(fname)
            #     files.append(''.join([n for n in os.path.basename(file_path) if n.isdigit()]))
            file_name= os.path.basename(file_path)
            df = pd.read_csv(file_path, sep=',')
            df.to_csv(f"{tables}/{subdir}/converted_storm_csv_{file_name}.csv")

def export_storms():
    section=['tropicalstorms']
    subdir='tropicalstorms'
    column_names = ['Year','Month' , 'TC number', 'Time step', 'Basin ID', 'Latitude', 'Longitude', 'Minimum pressure', 'Maximum wind speed', 'Radius to maximum winds', 'Category', 'Landfall', 'Distance to land']
    for subdir in section:
        file_list = glob.glob(os.path.join( os.path.join(data, subdir), "converted_storm_csv_STORM_DATA_*.txt.csv"))
        files = []
        appended_data=[]
        for file_path in file_list:
            file_name= os.path.basename(file_path)
            file_name = os.path.splitext(file_name)[0]
            df_points= pd.read_csv(file_path,  names=column_names, header=None)
            # print("Read file:" , file_path)
            for lat,lon in zip(df_points.Latitude, df_points.Longitude):
                if check(lon, lat):
                    files.append(file_path)
                    print(check(lon, lat))
                    sub_df = df_points[(df_points["Latitude"]==lat) & (df_points["Longitude"]==lon)]
                    appended_data.append(sub_df)
    appended_data = pd.concat(appended_data)
    # write DataFrame to an excel sheet 
    appended_data.to_csv(f"{tables}/{country}_{city}_{subdir}_storms.csv")
    return appended_data

def map_storms(appended_data):
    lon =appended_data["Longitude"]
    lat =appended_data["Latitude"]
    # lat = [28.6877899169922, 28.663863, 28.648287, 28.5429172515869]
    geometry = [Point(xy) for xy in zip(lon,lat)]
    wardlink = shp
    ward = gpd.read_file(wardlink, bbox=None, mask=None, rows=None)
    geo_df = gpd.GeoDataFrame(geometry = geometry)

    ward.crs = {'init':"epsg:4326"}
    geo_df.crs = {'init':"epsg:4326"}

    # plot the polygon
    ax = ward.plot(alpha=0.35, color='#d66058', zorder=1)
    # plot the boundary only (without fill), just uncomment
    #ax = gpd.GeoSeries(ward.to_crs(epsg=3857)['geometry'].unary_union).boundary.plot(ax=ax, alpha=0.5, color="#ed2518",zorder=2)
    ax = gpd.GeoSeries(ward['geometry'].unary_union).boundary.plot(ax=ax, alpha=0.5, color="#ed2518",zorder=2)

    # plot the marker
    ax = geo_df.plot(ax = ax, markersize = 20, color = 'red',\
                    marker = '*', zorder=3,
                    legend=True,  # Add legend 
                    legend_kwds={'loc':'upper right', 
                                    'bbox_to_anchor':(1, 1), 
                                                    'markerscale':1.01, 
                                                    'title_fontsize':'small', 
                                                    'fontsize':'x-small'
                                                    } ,
                                                    )
    ctx.add_basemap(ax, crs=geo_df.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    plt.xticks([])
    plt.yticks([])
        
    leg1 = ax.get_legend()
    # Set markers to square shape
    # for ea in leg1.legendHandles:
    #     ea.set_marker('s')
    # leg1.set_title(f'{legend_title}')
    # ax.title.set_text(f'{title}')
    # plt.tight_layout()
    # ax.figure.savefig(map_output)
    plt.show()


def process_gdp_iiasa(country):
    # section='tropicalstorms'
    for subdir in section:
        file_list = glob.glob(os.path.join( os.path.join(data, subdir), "iamc_db_GDP.xlsx"))
        for file_path in file_list:
            file_name= os.path.basename(file_path)
            file_name = os.path.splitext(file_name)[0]
            iiasa_gdp_df = pd.read_excel(file_path, sheet_name="data")
            iiasa_gdp_df = iiasa_gdp_df[iiasa_gdp_df["Region"]==country]
    return iiasa_gdp_df

def process_pop_iiasa(country):
    # section
    for subdir in section:
        file_list = glob.glob(os.path.join( os.path.join(data, subdir), "iamc_db.xlsx"))
        for file_path in file_list:
            file_name= os.path.basename(file_path)
            file_name = os.path.splitext(file_name)[0]
            iiasa_pop_df = pd.read_excel(file_path, sheet_name="data")
            iiasa_pop_df = iiasa_pop_df[iiasa_pop_df["Region"]==country]
    return iiasa_pop_df


def visualize_gdp_graph(df_for_graph, country, city, subdir):
    df_for_graph['Year'] = pd.to_datetime(
                            df_for_graph['Year'],
                            format='%Y'
                            )
    # plot data
    g = sns.lineplot(data=df_for_graph, x="Year", y="value", 
                     hue="Scenario"  , linewidth=2.5
                     )
    g.set(xlabel='Year', ylabel='GDP (Billions USD)')
    g.axes.yaxis.set_major_formatter(ticker.FuncFormatter(billions_formatter))
    # Tell matplotlib to interpret the x-axis values as dates
    g.xaxis_date()
    g.get_figure().savefig(f"{maps}/{country}_{city}_{subdir}_graph.jpeg" , dpi=300, bbox_inches='tight')

# iiasa_pop_df=process_pop_iiasa(country=country)
# # df_tranformed= iiasa_pop_df.pivot(index=['Scenario'], columns='Year', values="sum")
# # df=iiasa_pop_df.drop([], axis=1)
# df = pd.melt(iiasa_pop_df, id_vars=["Model", "Scenario"  ], value_vars='year')
# # df_graph = df.melt(ignore_index=False).reset_index()
# # iiasa_pop_df
# iiasa_gdp_df=process_gdp_iiasa(country=country)
# iiasa_gdp_df


def process_erosion():
    # section
    column_names = ['Latitude','Longitude' , 'percentile_1', 'percentile_5', 'percentile_17', 'percentile_50','percentile_83', 'percentile_95', 'percentile_99']
    for subdir in section:      
        file_list = glob.glob(os.path.join( os.path.join(data, subdir), "globalErosionProjections_Long_Term_Change*.csv"))
        files = []
        appended_data=[]
        for file_path in file_list:
            file_name= os.path.basename(file_path)
            file_name = os.path.splitext(file_name)[0]
            # df_points= pd.read_csv(file_path)
            df_points= pd.read_csv(file_path,  names=column_names, header=None)
            # print("Read file:" , file_path)
            for lat,lon in zip(df_points.Latitude, df_points.Longitude):
                if check(lon, lat):
                    files.append(file_path)
                    # print(check(lon, lat))
                    sub_df = df_points[(df_points["Latitude"]==lat) & (df_points["Longitude"]==lon)]
                    appended_data.append(sub_df)
    appended_data = pd.concat(appended_data)
    # write DataFrame to an excel sheet 
    appended_data.to_csv(f"{tables}/{country}_{city}_{subdir}_erosion.csv")
    erosion_df=appended_data
    return erosion_df
    # return df_points

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory


def difference_rasters(base_year_raster, end_year_raster, diff_out_path):
    r1=rioxarray.open_rasterio(base_year_raster, masked=True).squeeze()
    r2=rioxarray.open_rasterio(end_year_raster, masked=True).squeeze()
    diff_raster = r2 - r1
    diff_raster.rio.to_raster(diff_out_path)
    return diff_out_path

def difference_rasters_float_32(base_year_raster, end_year_raster, diff_out_path):
    r1=rioxarray.open_rasterio(base_year_raster, masked=True).squeeze()
    r2=rioxarray.open_rasterio(end_year_raster, masked=True).squeeze()
    diff_raster = r2 - r1
    diff_raster =diff_raster.astype(np.float32)
    diff_raster.rio.to_raster(diff_out_path)
    return diff_out_path


def convert_raster_to_float_32(input_raster, out_raster):
    input_raster=rioxarray.open_rasterio(input_raster, masked=True).squeeze()
    input_raster =input_raster.astype(np.float32)
    input_raster.rio.to_raster(out_raster)
    return out_raster



In [5]:
# def get_country_iso_code_from_country_name(country_name):
#     # get country iso_code from country name
#     gc_city_folder = os.path.join(data, f'{section[0]}/GC_countries/')
#     # Step 0.2: Get city's historical demographic change
#     df = pd.read_csv(gc_city_folder+'GC_{}.csv'.format(country_iso3))

#     if len(ssp_country)==0:
#         print('Country not found in SSP database. Check if ISO3 name of the country is correct, and whether the country is actually available in the SSP database.')
#         return 0,0
#     else:

In [6]:
def get_ssp2_2013_dataset(country_iso3, section, dataset,var):
    gc_city_folder = os.path.join(data, f'{section[0]}/GC_countries/')
    ssp_master_fn = os.path.join(data, f'{section[0]}/{dataset}')
    # Step 0.1: Get country-level projections
    if ".csv" in dataset:
        ssp_master = pd.read_csv(ssp_master_fn)
    print(ssp_master.columns)
    # ssp_master[ssp_master.columns] = ssp_master[ssp_master.columns].astype(str).apply(lambda col: col.str.upper())
    ssp_master.columns = [col.upper() for col in ssp_master if col in ssp_master.columns]
    # ssp_master['REGION'] = ssp_master['REGION'].str[:4]
    ssp_master['SCENARIO'] = ssp_master['SCENARIO'].str[:4]
    print("capital---", ssp_master.columns)
    ssp_country = ssp_master.loc[ssp_master['REGION']==country_iso3]
    print(f"variables----->>>> {ssp_country['MODEL'].unique()}")

    ssp_country = ssp_country.loc[ssp_country['MODEL']==var]
    ssp_country.reset_index(inplace=True, drop=True)
    if len(ssp_country)==0:
        print('Country not found in SSP database. Check if ISO3 name of the country is correct, and whether the country is actually available in the SSP database.')
        return 0,0
    else:
        del ssp_master
    
    
    # Step 0.2: Get city's historical demographic change
    global_cities = pd.read_csv(gc_city_folder+'GC_{}.csv'.format(country_iso3))
    gc_city = global_cities.loc[global_cities['Location']==city]
    if len(gc_city)==0:
        print('City not found in the Global Cities database. Manually check if the city name spelling is correct, and whether the city is actually available in the Global Cities database.')
        return 0,0
    else:
        del global_cities
    iso3=gc_city['iso3'].unique()[0]
    country_name=gc_city['Country'].unique()[0]
    print(iso3, country_name.title())
    ssp_country['iso3']=iso3.title()
    ssp_country['REGION']=country_name.title()
    
    return ssp_country, country_name, country_iso3

# 2023 ssp3
def get_ssp3_2023_dataset(country, section, dataset,var):
    ssp_master_fn = os.path.join(data, f'{section[0]}/{dataset}')
    # Step 0.1: Get country-level projections
    if ".csv" in dataset:
        ssp_master = pd.read_csv(ssp_master_fn)
    print(ssp_master.columns)
    ssp_master.columns = [col.upper() for col in ssp_master if col in ssp_master.columns]
    print("capital---", ssp_master.columns)
    ssp_country = ssp_master.loc[ssp_master['REGION']==country]
    ssp_country = ssp_country.loc[ssp_country['MODEL']==var]
    ssp_country.reset_index(inplace=True, drop=True)
    if len(ssp_country)==0:
        print('Country not found in SSP database. Check if ISO3 name of the country is correct, and whether the country is actually available in the SSP database.')
        return 0,0
    else:
        del ssp_master
    return ssp_country 

In [7]:
%%capture
country_iso3 = 'BGD'
var= 'IIASA-WiC POP' # 'IIASA GDP 2023', 
var_proper="Population"
section= ['demographic']
dataset=r"SspDb_country_data_2013-06-12.csv"
ssp_country_2013, country_name, country_iso3=get_ssp2_2013_dataset(country_iso3, section, dataset,var) 
ssp_country_2013=ssp_country_2013[ssp_country_2013['VARIABLE']==var_proper]

country_iso3 = 'BGD'
country_name="Bangladesh"
var= 'IIASA-WiC POP 2023' # 'IIASA GDP 2023', 
# var= 'IIASA GDP 2023'
section= ['demographic']
dataset=r"1706548837040-ssp_basic_drivers_release_3.0_full.csv"
ssp_country=get_ssp3_2023_dataset(country=country_name, 
                                                  section=section, dataset=dataset, var=var) 
ssp_country=ssp_country[ssp_country['VARIABLE']==var_proper]
df_merged=ssp_country_2013.merge(ssp_country, how='outer', 
                                 on=['SCENARIO','REGION','VARIABLE'], 
                                 suffixes=('_2013', '_2023'),
                                 indicator= True)
cols = df_merged.columns.tolist()
for j, s in enumerate(cols):  
    for i in range(2025,2105,5):
        try: 
            a = int(s) 
            y1=f"{i}_2013"
            y2=f"{i}_2023"
            # print(f"y1={y1}, y2={y2}, s={s}")
            df_merged[f"rescaling_factor_{y2}_div_{y1}"] = df_merged[y2]/df_merged[y1]
            df_merged[f"updated_{i}"] = df_merged[y1]*df_merged[f"rescaling_factor_{y2}_div_{y1}"]
            df_merged.to_csv()
        except ValueError: 
                    print(f'{s} was not an integer')                                 
df_merged.to_csv(f"{tables}/{country}_{city}_{var}_rescaled.csv")
df_merged


Combine by SCENARIO	REGION	VARIABLE the two dataset. Name the columns with left and right. This will give us the scaling facor

In [8]:
%%capture
country_iso3 = 'BGD'
var= 'IIASA GDP' #'IIASA-WiC POP' # 'IIASA GDP 2023', 
var_proper="GDP|PPP" #"Population"
section= ['demographic']
dataset=r"SspDb_country_data_2013-06-12.csv"
ssp_country_2013, country_name, country_iso3=get_ssp2_2013_dataset(country_iso3, section, dataset,var) 
ssp_country_2013=ssp_country_2013[ssp_country_2013['VARIABLE']==var_proper]

# Get gdp defkator and inflate the gdp to 2017 USD 
dataset=r'gdp_deflator.csv'
deflator= os.path.join(data, f'{section[0]}/{dataset}')
deflator = pd.read_csv(deflator)
deflator=deflator[deflator["Country Code"]==country_iso3]
deflator_multiplier=(deflator['2017']/deflator['2005']).item()
print(deflator_multiplier)

# Multiply all columns by the defaltor multiplier
cols = ssp_country_2013.columns.tolist()
for j, s in enumerate(cols):  
    for i in range(2025,2105,5):
        try: 
            a = int(s) 
            y1=f"{i}"
            # print(f"y1={y1}, y2={y2}, s={s}")
            ssp_country_2013[f"{i}"] = ssp_country_2013[y1]*deflator_multiplier
        except ValueError: 
                    print(f'{s} was not an integer') 
country_name="Bangladesh"
var= 'IIASA GDP 2023'
section= ['demographic']
dataset=r"1706548837040-ssp_basic_drivers_release_3.0_full.csv"
ssp_country=get_ssp3_2023_dataset(country=country_name, 
                                                  section=section, dataset=dataset, var=var) 

df_merged=ssp_country_2013.merge(ssp_country, how='outer', 
                                 on=['SCENARIO','REGION','VARIABLE'], 
                                 suffixes=('_2013', '_2023'),
                                 indicator= True)

cols = df_merged.columns.tolist()
for j, s in enumerate(cols):  
    for i in range(2025,2105,5):
        try: 
            a = int(s) 
            y1=f"{i}_2013"
            y2=f"{i}_2023"
            # print(f"y1={y1}, y2={y2}, s={s}")
            df_merged[f"rescaling_factor_{y2}_div_{y1}"] = df_merged[y2]/df_merged[y1]
            df_merged[f"updated_{i}"] = df_merged[y1]*df_merged[f"rescaling_factor_{y2}_div_{y1}"]
        except ValueError: 
                    print(f'{s} was not an integer')                                 
df_merged.to_csv(f"{tables}/{country}_{city}_{var}_rescaled.csv")

In [9]:
def get_rescaling_factor(input_csv, year, ssp_value):
    df= pd.read_csv(input_csv)
    df=df[[f"rescaling_factor_{year}_2023_div_{year}_2013", "SCENARIO"]] 
    rescaling_factor=df[df['SCENARIO']==ssp_value][f"rescaling_factor_{year}_2023_div_{year}_2013"].item()
    print(f"year--> {year}  ....ssp_value-->> {ssp_value}...rescaling_factor--->> {rescaling_factor}")
    return rescaling_factor

var='IIASA GDP 2023'
input_csv=f"{tables}/{country}_{city}_{var}_rescaled.csv"
year=2025
ssp_value="SSP1"
rescaling_factor=get_rescaling_factor(input_csv, year, ssp_value)
rescaling_factor



year--> 2025  ....ssp_value-->> SSP1...rescaling_factor--->> 2.7084350327164718e-17


2.7084350327164718e-17

In [10]:

var='IIASA GDP 2023'
input_csv=f"{tables}/{country}_{city}_{var}_rescaled.csv"
year=2025
ssp_value="SSP1"
rescaling_factor=get_rescaling_factor(input_csv, year, ssp_value)
rescaling_factor



year--> 2025  ....ssp_value-->> SSP1...rescaling_factor--->> 2.7084350327164718e-17


2.7084350327164718e-17

In [11]:
var= 'IIASA-WiC POP 2023' # 'IIASA GDP 2023', 
input_csv=f"{tables}/{country}_{city}_{var}_rescaled.csv"
year=2025
ssp_value="SSP1"
rescaling_factor=get_rescaling_factor(input_csv, year, ssp_value)
rescaling_factor

year--> 2025  ....ssp_value-->> SSP1...rescaling_factor--->> 1.031974297504538


1.031974297504538

Create a categorical map function with same legend breaks across maps

In [12]:
def export_popdynamics():
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name.split('_')[1].upper() 
                    ssp = file_name.split('_')[0].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    # print(file_name,  file_name_path, subdir, ssp, Year)
                    combined_results.append(df)
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp=="SSP2" or ssp=="SSP5") and (Year=="2050" or Year=="2100"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Population in {Year} under {ssp}"
                        legend_title="Population (x1000)"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
                        
                        var= 'IIASA-WiC POP 2023' # 'IIASA GDP 2023', 
                        input_csv=f"{tables}/{country}_{city}_{var}_rescaled.csv"
                        year=Year #2025
                        ssp_value= ssp # "SSP1"
                        rescaling_factor=get_rescaling_factor(input_csv, year, ssp_value)
                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].multiply(rescaling_factor)
                        driver= 'ESRI Shapefile'
                        polygonized_shapefile.to_file(clipped_output_shapefile, driver)

                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(1000).round(0)
                        # Function for getting quantile ranges. 
                        polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 )
                        a = pd.qcut(polygonized_shapefile[raster_val], 5).cat.categories.right
                        b = pd.qcut(polygonized_shapefile[raster_val], 5).cat.categories.left
                        binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
                        
                        # binned = pd.cut(polygonized_shapefile[raster_val], bins=bins, labels=labels)
                        # print(f"a:{a}---df1:{df1} df2:{df2} , bins: {bins}, binned: {binned}, binned_df {binned_df}" )

                        map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles.png"
                        quantile_range= "quantile_range"
                        export_the_map= export_map_categorical(vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                                crs=crs, cmap=cmap , visualize_column=quantile_range,  \
                                title=map_title, map_output=map_output)

                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                                crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)


        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
        # Wide
        [stats_to_get] = stats_to_get
        df_tranformed= df.pivot(index=['Scenario'], columns='Year', values=stats_to_get)
        # Long
        df_graph = df_tranformed.melt(ignore_index=False).reset_index()
        # export csv of wide
        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
        #run the visualize function to export the heatmap
        visualize_popdynamics_figure(df_tranformed, country, city, subdir)
        # Visualize pop dynamics graph
        visualize_popdynamics_graph(df_graph, country, city, subdir)
        # Merge json files for all ssps 
        merge_JsonFiles_=merge_JsonFiles(dir=shapefiles, section=subdir)
        return clipped_output_shapefile 

In [13]:
def export_popdynamics_change(input_ssp, start_year, end_year):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        extension = '.tif'
        two_raster_list=[]
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name.split('_')[1].upper() 
                    ssp = file_name.split('_')[0].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp==f"{input_ssp}" and Year==f"{start_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = out_rst #f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Difference in Population in {Year} under {ssp} from 2050"
                        legend_title="Population (x1000)"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        
                        two_raster_list.append(projected_raster)

                    elif (ssp==f"{input_ssp}" ) and (  Year==f"{end_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}_1.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 =out_rst # f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        # map_title= f"Population in {Year} under {ssp}"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}_1.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        two_raster_list.append(projected_raster)

        # for delta in len(two_raster_list):
        diff_out_path = f'{rasters}/projected_clipped_difference_{subdir}_{file_name}.tif'
        base_year_raster=two_raster_list[0]
        end_year_raster=two_raster_list[1]
        print(f"base_year_raster:: {base_year_raster}-----end_year_raster::{end_year_raster}")
        diff_out_path=difference_rasters(base_year_raster, end_year_raster, diff_out_path)
                                
        legend_title="Population(x1000)"
        map_title= f"Population change during {start_year}-{end_year} ({input_ssp})"
        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}_diff.shp'
        output_geojson= f'{shapefiles}/geojson_{subdir}_diff.geojson'
        output_csv= f'{tables}/csv_{subdir}_{file_name}_diff.csv'
        driver= 'ESRI Shapefile'
        mask_value=None
        raster_val= subdir[0:9]
        name = polygonize_with_negatives(raster_file=diff_out_path ,crs=EPSG_str, output_shapefile=output_shapefile, \
                            driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_difference_{subdir}_{file_name}.shp'
        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                    output_geojson= output_geojson, output_csv=output_csv, \
                                                    clipped_output_shapefile=clipped_output_shapefile)                        
        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(1000).round(0)
        
        # Function for getting quantile ranges. 
        polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 ,duplicates='drop')
        a = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.right
        b = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.left
        binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
        
        map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles_diff_{input_ssp}_{start_year}_{end_year}.png"
        quantile_range= "quantile_range"
        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=raster_val,  \
                title=map_title, map_output=map_output)
       
        return polygonized_shapefile 

In [14]:
def export_gdp():
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    Year = file_name[3:7].upper() 
                    ssp = file_name[8:14].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    # print(file_name,  file_name_path, subdir, ssp, Year)
                    combined_results.append(df)
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp=="SSP2" or ssp=="SSP5") and (Year=="2050" or Year=="2100"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"GDP(Millions USD) in {Year} under {ssp}"
                        legend_title="GDP(Millions USD)"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
                        
                        var='IIASA GDP 2023'
                        input_csv=f"{tables}/{country}_{city}_{var}_rescaled.csv"
                        year=Year #2025
                        ssp_value= ssp # "SSP1"
                        rescaling_factor=get_rescaling_factor(input_csv, year, ssp_value)
                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].multiply(rescaling_factor)
                        driver= 'ESRI Shapefile'
                        polygonized_shapefile.to_file(clipped_output_shapefile, driver)


                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(1000000).round(2)
                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)


        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
       # Wide
        [stats_to_get] = stats_to_get
        df_tranformed= df.pivot(index=['Scenario'], columns='Year', values=stats_to_get)        
        # Long
        df_graph = df_tranformed.melt(ignore_index=False).reset_index()
        # export csv of wide
        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
        #run the visualize function to export the heatmap
        visualize_gdp_figure(df_tranformed, country, city, subdir)
        visualize_gdp_graph(df_graph, country, city, subdir)
        merge_JsonFiles_=merge_JsonFiles(dir=shapefiles, section=subdir)

In [15]:
def export_gdp_change(input_ssp, start_year, end_year):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        extension = '.tif'
        two_raster_list=[]
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name[3:7].upper() 
                    ssp = file_name[8:14].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp==f"{input_ssp}" and Year==f"{start_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = out_rst #f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        
                        two_raster_list.append(projected_raster)

                    elif (ssp==f"{input_ssp}" ) and (  Year==f"{end_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}_1.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 =out_rst # f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        # map_title= f"Population in {Year} under {ssp}"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}_1.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        two_raster_list.append(projected_raster)

        # for delta in len(two_raster_list):
        diff_out_path = f'{rasters}/projected_clipped_difference_{subdir}_{file_name}.tif'
        base_year_raster=two_raster_list[0]
        end_year_raster=two_raster_list[1]
        print(f"base_year_raster:: {base_year_raster}-----end_year_raster::{end_year_raster}")
        diff_out_path=difference_rasters(base_year_raster, end_year_raster, diff_out_path)
        
        map_title= f"GDP(Millions USD) change during {start_year}-{end_year} ({input_ssp})"
        legend_title="GDP(Millions USD)"
        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}_diff.shp'
        output_geojson= f'{shapefiles}/geojson_{subdir}_diff.geojson'
        output_csv= f'{tables}/csv_{subdir}_{file_name}_diff.csv'
        driver= 'ESRI Shapefile'
        mask_value=None
        raster_val= subdir[0:9]
        name = polygonize_with_negatives(raster_file=diff_out_path ,crs=EPSG_str, output_shapefile=output_shapefile, \
                            driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_difference_{subdir}_{file_name}.shp'
        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                    output_geojson= output_geojson, output_csv=output_csv, \
                                                    clipped_output_shapefile=clipped_output_shapefile)                        
        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(1000000).round(2)

        # polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(1000).round(0)
        
        # Function for getting quantile ranges. 
        polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 ,duplicates='drop')
        a = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.right
        b = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.left
        binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
        
        map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles_diff_{input_ssp}_{start_year}_{end_year}.png"
        quantile_range= "quantile_range"
        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=raster_val,  \
                title=map_title, map_output=map_output)
       
        return polygonized_shapefile 
         

## Urban Land Extent
The dataset includes gridded global maps of urban land fractions for the period of 2000–2100 at the 1-km resolution and updates at decadal time intervals, under five urban land expansion scenarios consistent with the SSPs (SSP1 – sustainability, SSP2 – middle of the road, SSP3 – regional rivalry, SSP4 – inequality, SSP5 – fossil-fuelled development)

In [16]:
def export_urbanland():
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    Year = file_name[5:9].upper() 
                    ssp = file_name[0:4].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    # print(file_name,  file_name_path, subdir, ssp, Year)
                    combined_results.append(df)
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp=="SSP2" or ssp=="SSP5") and (Year=="2050" or Year=="2100"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"%Urban Land in {Year} under {ssp}"
                        legend_title="%age Urban Land"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].multiply(100).round(0)
                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)

        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
        # Wide
        # df_tranformed= df.pivot(index=['Scenario'], columns='Year', values='mean')
        [stats_to_get] = stats_to_get
        df_tranformed= df.pivot(index=['Scenario'], columns='Year', values=stats_to_get)
        # Long
        df_graph = df_tranformed.melt(ignore_index=False).reset_index()
        # export csv of wide
        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
        # print(df_tranformed)
        #run the visualize function to export the heatmap
        visualize_urbanland_figure(df_tranformed, country, city, subdir)
        visualize_urbanland_graph(df_graph, country, city, subdir)
        merge_JsonFiles_=merge_JsonFiles(dir=shapefiles, section=subdir)

In [17]:
def export_urbanland_change(input_ssp, start_year, end_year):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        extension = '.tif'
        two_raster_list=[]
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name.split('_')[1].upper() 
                    ssp = file_name.split('_')[0].upper() 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp==f"{input_ssp}" and Year==f"{start_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = out_rst #f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Difference in Population in {Year} under {ssp} from 2050"
                        legend_title="Population (x1000)"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        
                        two_raster_list.append(projected_raster)

                    elif (ssp==f"{input_ssp}" ) and (  Year==f"{end_year}"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}_1.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 =out_rst # f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        # map_title= f"Population in {Year} under {ssp}"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}_1.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        two_raster_list.append(projected_raster)

        # for delta in len(two_raster_list):
        diff_out_path = f'{rasters}/projected_clipped_difference_{subdir}_{file_name}.tif'
        base_year_raster=two_raster_list[0]
        end_year_raster=two_raster_list[1]
        print(f"base_year_raster:: {base_year_raster}-----end_year_raster::{end_year_raster}")
        diff_out_path=difference_rasters(base_year_raster, end_year_raster, diff_out_path)
                                
        map_title= f"%Urban Land change during {start_year}-{end_year}({input_ssp})"
        legend_title="%age Urban Land"

        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}_diff.shp'
        output_geojson= f'{shapefiles}/geojson_{subdir}_diff.geojson'
        output_csv= f'{tables}/csv_{subdir}_{file_name}_diff.csv'
        driver= 'ESRI Shapefile'
        mask_value=None
        raster_val= subdir[0:9]
        name = polygonize_with_negatives(raster_file=diff_out_path ,crs=EPSG_str, output_shapefile=output_shapefile, \
                            driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_difference_{subdir}_{file_name}.shp'
        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                    output_geojson= output_geojson, output_csv=output_csv, \
                                                    clipped_output_shapefile=clipped_output_shapefile)                        
        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
        
        # Function for getting quantile ranges. 
        polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 ,duplicates='drop')
        a = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.right
        b = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.left
        binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
        
        map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles_diff_{input_ssp}_{start_year}_{end_year}.png"
        quantile_range= "quantile_range"
        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=raster_val,  \
                title=map_title, map_output=map_output)
       
        return polygonized_shapefile 

###                                Hourly anthropogenic heat flux

Under this assumption, annual-average anthropogenic heat of the country (Qf−y) is assumed to be equal to the sum of the primary energy consumption and anthropogenic heat from metabolic processes (QM). Primary energy consumption of the country can be further partitioned to components such as “heat loss” (QL), and AHE of “industrial and agricultural” (QIA), and “commercial, residential, and transport sectors” (QM) (Eq. 1).

Qf−y=QL+QCRT+QIA+QM

Data available: temporal:2050 only, Scenarios: SSP 3 only. Both Hourly, monthly and yearly averages are available here: https://urbanclimate.tse.ens.titech.ac.jp/database/AHE/AH4GUC/future/
Hourly data is avaiable for 24 hours of each month. Total hourly tiffs: 24*12*1 (data is available for 2010 and 2050)

We show maps for the 12 months for year 2050.  We can produce these for each hour of a given month.



In [18]:
# heatflux
def export_heatflux():
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # AHE_2010_01_average
                    Year = file_name.split('_')[1].upper() 
                    Month = file_name.split('_')[2].upper() 
                    ssp = "SSP3" #Only fo this SSP we have AHE  
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    df['Month'] = Month
                    # print(file_name,  file_name_path, subdir, ssp, Year)
                    combined_results.append(df)
                    #clip and export rasters for the pertinent ssp and year combo
                    if  (Year=="2050" and Month=="YEAR" ):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Annualized average Urban Heat Flux \n in Year:{Year} under {ssp}"
                        legend_title="UHF(W/sq.m)"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs,  unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(100).round(1)
                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                    if  (Year=="2050" and Month!="YEAR" ):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Urban Heat Flux during \n Month-Year:{Month}-{Year} under {ssp}"
                        legend_title="UHF(W/sq.m)"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
                        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(100).round(1)
                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
                                title=map_title, map_output=map_output)

        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
        # Wide
        # print(df)
        [stats_to_get] = stats_to_get
        # df_tranformed= df.pivot(index=['Scenario'], columns='Year', values=stats_to_get)
        df_tranformed= df.pivot_table(index=['Scenario'] , columns='Year', values=stats_to_get, aggfunc=stats_to_get)        
        # Long
        df_graph = df_tranformed.melt(ignore_index=False).reset_index()
        # export csv of wide
        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
        merge_JsonFiles_=merge_JsonFiles(dir=shapefiles, section=subdir)
        

In [19]:
def export_heatflux_change(input_ssp, start_year, end_year):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        extension = '.tif'
        two_raster_list=[]
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name.split('_')[1].upper() 
                    Month = file_name.split('_')[2].upper() 
                    ssp = "SSP3" #Only fo this SSP we have AHE  
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    df['Month'] = Month
                    #clip and export rasters for the pertinent ssp and year combo

                    if (ssp==f"{input_ssp}" and Year==f"{start_year}" and Month=="YEAR"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = out_rst #f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        
                        two_raster_list.append(projected_raster)

                    elif (ssp==f"{input_ssp}" ) and (  Year==f"{end_year}" and Month=="YEAR"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}_1.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 =out_rst # f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        # map_title= f"Population in {Year} under {ssp}"
                        [statistic]= stats_to_get
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}_1.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        two_raster_list.append(projected_raster)

        # for delta in len(two_raster_list):
        diff_out_path = f'{rasters}/projected_clipped_difference_{subdir}_{file_name}.tif'
        base_year_raster=two_raster_list[0]
        end_year_raster=two_raster_list[1]
        print(f"base_year_raster:: {base_year_raster}-----end_year_raster::{end_year_raster}")
        diff_out_path=difference_rasters_float_32(base_year_raster, end_year_raster, diff_out_path)
                                
        map_title= f"Change in average Urban Heat Flux(annual) \n  during {start_year}-{end_year}({input_ssp})"
        legend_title="UHF(W/sq.m)"
        
        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}_diff.shp'
        output_geojson= f'{shapefiles}/geojson_{subdir}_diff.geojson'
        output_csv= f'{tables}/csv_{subdir}_{file_name}_diff.csv'
        driver= 'ESRI Shapefile'
        mask_value=None
        raster_val= subdir[0:9]
        name = polygonize_with_negatives(raster_file=diff_out_path ,crs=EPSG_str, output_shapefile=output_shapefile, \
                            driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_difference_{subdir}_{file_name}.shp'
        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                    output_geojson= output_geojson, output_csv=output_csv, \
                                                    clipped_output_shapefile=clipped_output_shapefile)                        
        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
        polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].div(100).round(1)

        # Function for getting quantile ranges. 
        polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 ,duplicates='drop')
        a = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.right
        b = pd.qcut(polygonized_shapefile[raster_val], 5,duplicates='drop').cat.categories.left
        binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
        
        map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles_diff_{input_ssp}_{start_year}_{end_year}.png"
        quantile_range= "quantile_range"
        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                crs=crs, cmap=cmap , visualize_column=raster_val,  \
                title=map_title, map_output=map_output)
       
        return polygonized_shapefile 

### UHI intensification
Snippet from the paper: Our results show that urban land expansion through 2050 will lead to warming of surface air temperature (hereafter referred to as 'urban expansion-warming'), which is about 40%–70% as strong as that caused by GHG emissions (hereafter referred to as 'GHG emissions-warming'; figure 2 and table 1). During summer, urban expansion will increase daily minimum and maximum temperatures by 0.5°C–0.7 °C on average (up to ∼3 °C in some locations), which is about 39%–64% (up to >200%) as strong as the GHG emissions-induced warming (multi-model ensemble average in the IPCC RCP 4.5 scenario). During winter, the urban expansion-warming will be weaker: on average 0.4 °C–0.6 °C (up to ∼2 °C), about 43%–65% (up to ∼170%) as strong as the GHG emissions-warming. Despite varying magnitudes of urban expansion, intensities of the subsequent warming are relatively similar across scenarios (table 1). 

Data available: temporal:2050 only, Scenarios: All SSPs and Day and Night Degrees Centigrade changes.


In [20]:
def export_urbanheatisland():
    hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    # print(file_name,  file_name_path, subdir)
                    Year = file_name.split('_')[1].title() 
                    Year=Year.replace("Nig", "Night") 
                    ssp = file_name.split('_')[0].upper()
                    ssp = ssp.split('-')[1].upper() 
                    season = file_name.split('_')[2].title() 
                    season=season.replace("Win", "Winter") 
                    season=season.replace("Sum", "Summer") 
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    raster = rasterio.open(rst)
                    # vector = load_shapefile(shp)
                    vector = load_shapefile_buffer(shp)
                    p_vector = reproject_gpdf(vector, raster)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    # print(f"season:{season} ssp:{ssp} Year or day and night:{Year}")
                    combined_results.append(df)
                    #clip and export rasters for the pertinent ssp and year combo
                    if (ssp=="SSP2" or ssp=="SSP5") and (Year=="Day" or Year=="Night"):
                        out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                        clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                        out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
                        map_title= f"Heat Island Intensification \n during {season} {Year} under {ssp}"
                        legend_title="HIU"
                        [statistic]= stats_to_get
                        hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
                            hexbins_projected=hexbins_projected , crs=WGS84 , \
                                raster=rst , out_raster=out_rst_2, statistic=statistic)
                        crs = 4326
                        projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                        unprojected_raster = out_rst_2
                        reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                           projected_raster=projected_raster)
                        # cmap = plt.get_cmap('RdYlGn', 10)
                        # cmap = truncate_colormap(cmap, 0.1, 1)
                        map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                        document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                        ###export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                        output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                        output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                        output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                        driver= 'ESRI Shapefile'
                        mask_value=None
                        raster_val= subdir[0:9]
                        name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                           driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                        clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                        clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                   output_geojson= output_geojson, output_csv=output_csv, \
                                                                    clipped_output_shapefile=clipped_output_shapefile)                        
                        polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
                        export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, \
                                    crs=crs, cmap=cmap , visualize_column=raster_val, \
                                        title=map_title, map_output=map_output)
                        export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
                                                           hexagons=polygonized_shapefile,legend_title=legend_title, \
                                                            legends_format=legends_format, crs=crs, cmap=cmap , \
                                                                visualize_column=raster_val,  \
                                                                    title=map_title, map_output=map_output)
        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
        # Wide
        [stats_to_get] = stats_to_get
        df_tranformed= df.pivot_table(index=['Scenario'] , columns='Year', values=stats_to_get, aggfunc=stats_to_get)        
        # Long
        df_graph = df_tranformed.melt(ignore_index=False).reset_index()
        # export csv of wide
        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")


Fathom Section

In [42]:
def run_fast_scandir(dir):    # dir: str, ext: list
    subfolders=[]
    for f in os.scandir(dir):
        if f.is_dir():
            subfolders.append(f.path)
    return subfolders


In [76]:
def create_fathom_mosaic(section):
    subdir=section[0]
    dir=Path(os.path.join(data, f"{subdir}"))
    for subdir in section:
        combined_results = []
        rasters_for_mosaic = defaultdict(list)
        extension = '.tif'
        for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    file_name_clean = file_name.split("_")[:-1]
                    file_name_clean = "_".join(file_name_clean)
                    rasters_for_mosaic[file_name_clean].append(file_name_path)

    output_dir=Path(os.path.join(data, f"{subdir}/clean"))
    if os.path.exists(output_dir):
        try:
            shutil.rmtree(output_dir, ignore_errors=True)
            print(f"deleted {output_dir}")
        except:
            print(f"Couldnt delete {output_dir}")
            pass
    
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    for key, value in rasters_for_mosaic.items():
        print(f"{key}: {value}")
        raster_to_mosiac=[]
        for p in value:
            raster = rio.open(p)
            raster_to_mosiac.append(raster)
            # print(f"input_dir {key} has raster>>>>{p}")
        
        # output_path= Path(os.path.join(output_dir)
        # if not os.path.exists(output_path):
        #     os.mkdir(output_path)
        output_path= os.path.join(output_dir, f"{key}.tif")
        print(f" file_name-->> {file_name}-raster_to_mosiac : {raster_to_mosiac}----output_path {output_path}")
        mosaic, output = merge(raster_to_mosiac)

        # copy raster's metadata and update it to match the height and width of the mosaic.
        output_meta = raster.meta.copy()
        output_meta.update(
            {"driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": output,
            }
        )

        # write the mosaiced file in the folder.
        with rio.open(output_path, 'w', **output_meta) as m:
            m.write(mosaic)

    return rasters_for_mosaic


In [130]:

def export_fathom(input_dir, section,shp, stat):
    vector_file = gpd.read_file(shp).to_crs(WGS84)
    for subdir in section:
        combined_results = []
        extension = '.tif'
        for root, dirs_list, files_list in os.walk(input_dir):
            for file_name in files_list:
                if os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    print(file_name,  file_name_path, subdir)
                    file_name_clean=file_name.replace("-", " ")
                    file_name_clean=file_name_clean.split(" ") #[0:3]

                    return_period=file_name_clean[0]
                    return_period=return_period.split("in")[1]
                    # print(file_name_clean, return_period)
                    flood_type= file_name_clean[1].title()
                    print(file_name_clean, return_period, flood_type)

                    var= file_name_clean[1]
                    Year= file_name_clean[3]
                    # scenario=file_name_clean[4]
                    # list_length=len(file_name_clean)
                    # if "SSP" in scenario:
                    if len(file_name_clean)>4:
                        # scenario=file_name_clean[4]
                        ssp=file_name_clean[4]
                        # ssp=ssp.replace("_", "-")
                    else:
                        ssp="Present"

                    title= f"{flood_type} flood in {Year} with a \n return period of {return_period} years ({ssp})"
                    # print(f"file_name  for legends----------------{file_name}---clean{file_name_clean}")
                    print(f"title->>>>>> {title}")

                            
                    file_name_projected= f'{file_name}_wsg84.tif'
                    file_name_projected= file_name_projected.replace("-", "_")
                    output_raster=os.path.join(root, file_name_projected) 
                    rst = file_name_path 
                    vector = load_shapefile_buffer(shp)
                    out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
                    clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
                    
                    raster = rasterio.open(rst)
                    p_vector = reproject_gpdf(vector, raster)
                    # vector = load_shapefile(shp)
                    stats_to_get = list_statistics(stat)
                    df = calculate_zonal_stats(p_vector, out_rst, stats_to_get)
                    df['Scenario'] = ssp
                    df['Year'] = Year
                    df['Flood type'] = flood_type
                    # print(file_name,  file_name_path, subdir, ssp, Year)
                    combined_results.append(df)
                    print(df.head())
        
                    map_title= f"{title}"
                    legend_title= f"Flood depth(cm)"
                    print(f"map_title {map_title}----legend_title-->>>{legend_title}")
                    crs = 4326
                    projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
                    unprojected_raster = out_rst
                    reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
                                                        projected_raster=projected_raster)
                    
                    map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
                    document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
                    #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
                    output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
                    output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
                    output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
                    driver= 'ESRI Shapefile'
                    mask_value=None
                    raster_val= subdir[0:9]
                    name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
                                        driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
                    clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
                    clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
                                                                output_geojson= output_geojson, output_csv=output_csv, \
                                                                clipped_output_shapefile=clipped_output_shapefile)                        
                    polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84)
                    # Function for getting quantile ranges. 
                    polygonized_shapefile["quantile_range"] = pd.qcut(polygonized_shapefile[raster_val],5 )
                    a = pd.qcut(polygonized_shapefile[raster_val], 5).cat.categories.right
                    b = pd.qcut(polygonized_shapefile[raster_val], 5).cat.categories.left
                    binned = pd.cut(polygonized_shapefile[raster_val], bins=b)
                    
                    # binned = pd.cut(polygonized_shapefile[raster_val], bins=bins, labels=labels)
                    # print(f"a:{a}---df1:{df1} df2:{df2} , bins: {bins}, binned: {binned}, binned_df {binned_df}" )
                    legends_format= '{:,.0f}'
                    cmap = "OrRd"
                    map_output = f"{maps}/{city}_{subdir}_{file_name}_quantiles.png"
                    quantile_range= "quantile_range"
                    export_the_map= export_map_categorical(vector_file=vector_file , \
                            hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                            crs=crs, cmap=cmap , visualize_column=quantile_range,  \
                            title=map_title, map_output=map_output)

                    # ------------------------
                    export_the_map= export_map(map_dir= output, vector_file=vector_file , \
                            hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format,  \
                            crs=crs, cmap=cmap , visualize_column=raster_val,  \
                            title=map_title, map_output=map_output)


        # Append vertically and then pivot horizontally.
        df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
        # Wide
        df_tranformed= df # df[['Flood type', 'Year' ,'Scenario', stats_to_get]] #df.pivot(index=['Flood type', 'Year' ], columns=['Scenario'], values=stats_to_get)
        print(f"df_tranformed NOT: {df_tranformed}")

        df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
        merge_JsonFiles_=merge_JsonFiles(dir=shapefiles, section=subdir)
        return clipped_output_shapefile , df_tranformed 

def fathom_barchart(df,subtitle, y_var):
    df['scenario_year']=df['Scenario']+ " ("+df['Year']+")"
    # Draw a nested barplot by flood-type and ssp
    g = sns.catplot(
        data=df, kind="bar",
        x='Flood type', y=y_var, hue='scenario_year',
        palette="dark", alpha=.6, height=6
    )
    g.fig.suptitle(f"Flood Projections {subtitle}")
    # title
    new_title = 'Scenario'
    g._legend.set_title(new_title)
    # sns.move_legend(g, "upper right", bbox_to_anchor=(.8, 1.02), frameon=False)
    g.set_axis_labels("Flood type", "Flood depth(cm)")
    g.figure.savefig(f"{maps}/{y_var}_fathom_flood_barchart.jpeg", dpi=300, bbox_inches='tight')



In [131]:
stat= [ 'mean','percentile_95']
section= ['fathom']

# rasters_for_mosaic=create_fathom_mosaic(section=section)
input_dir=PureWindowsPath(os.path.join(data, f"{section[0]}/clean"))
clipped_output_shapefile , df_tranformed =export_fathom(input_dir, section,shp, stat)

# titles= ["(Average)", "(95 Percentile)"]
# for s, t in zip(stat,titles):  
#     fathom_barchart(df=df_tranformed, subtitle= t, y_var=s)


1in10-COASTAL-DEFENDED-2030-SSP1 C:\Users\Aziz\Dropbox\CRP\FCS\data\fathom\clean\1in10-COASTAL-DEFENDED-2030-SSP1.tif fathom
['1in10', 'COASTAL', 'DEFENDED', '2030', 'SSP1'] 10 Coastal
title->>>>>> Coastal flood in 2030 with a 
 return period of 10 years (SSP1)
                                            geometry  mean  percentile_95  \
0  POLYGON ((91.73066 22.35957, 91.73024 22.36080...  0.65            0.0   

  Scenario  Year Flood type  
0     SSP1  2030    Coastal  
map_title Coastal flood in 2030 with a 
 return period of 10 years (SSP1)----legend_title-->>>Flood depth(cm)
1in10-COASTAL-DEFENDED-2030-SSP2 C:\Users\Aziz\Dropbox\CRP\FCS\data\fathom\clean\1in10-COASTAL-DEFENDED-2030-SSP2.tif fathom
['1in10', 'COASTAL', 'DEFENDED', '2030', 'SSP2'] 10 Coastal
title->>>>>> Coastal flood in 2030 with a 
 return period of 10 years (SSP2)
                                            geometry  mean  percentile_95  \
0  POLYGON ((91.73066 22.35957, 91.73024 22.36080...  0.65            0.0 

In [69]:
def export_map_fathom_overlays(vector_file,  layer_1, layer,legend_title,legends_format,legends_format_1,legend_label,legend_label_1, crs ,cmap,cmap_1,  visualize_column,visualize_column_1, title, map_output):
   #  fig = plt.figure(figsize=(width, height))
   height=7
   width=8
   fig, ax = plt.subplots(figsize=(width, height))
   #  ax = fig.add_subplot(111)
   ax = layer_1.to_crs(crs).plot(ax=ax, 
                                 column= visualize_column_1, \
                                 linewidth=0 , 
                                 alpha=0.6, 
                              #    scheme="quantiles", 
                                 cmap=cmap_1, 
                                 label=f"{legend_label_1}" ,
                                       )

   ax = layer.to_crs(crs).plot(ax=ax, 
                                 column= visualize_column, 
                                 alpha=0, 
                                 facecolor='none',
                                 scheme="quantiles", 
                                 cmap=cmap,
                                 label=f"{legend_label}" ,
                                 legend=False 
                                       )
   layer_filtered=layer[layer[visualize_column]>0]
   ax = layer_filtered.to_crs(crs).plot(ax=ax, 
                                 column= visualize_column, 
                                 alpha=.6, 
                              #    hatch="/",
                                 scheme="quantiles", 
                                 cmap=cmap,
                                 label=f"{legend_label}" ,
                                 legend=False 
                                       )

   vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
   
   cx.add_basemap(ax, 
               source=cx.providers.Esri.WorldImagery,  
               crs=crs,
               attribution=False, #No citation
               ) 

   visualize_column_1=visualize_column_1 #"popdynami"
   layer_1["quantile_range"] = pd.qcut(layer_1[visualize_column_1],5 )
   a = pd.qcut(layer_1[visualize_column_1], 5).cat.categories.right
   b = pd.qcut(layer_1[visualize_column_1], 5).cat.categories.left
   c=[str(int(m))+ "-" +str(int(n)) for m,n in zip(b ,a)]

   cmap_1 = cm.get_cmap(cmap_1)
   # title_patch = mpatches.Patch(color='none',label=f'{legend_label_1}')
   # title_patch = mpatches.Patch(label=f'{legend_label_1}')
   title_patch = Line2D([0], [0], marker='o', color='none', label=f'{legend_label_1}', markerfacecolor='none', markersize=1)
   white_patch = mpatches.Patch(color=cmap_1(0.0), label=c[0])
   lowblue_patch = mpatches.Patch(color=cmap_1(0.25), label=c[1])
   midblue_patch = mpatches.Patch(color=cmap_1(0.5), label=c[2])
   highblue_patch = mpatches.Patch(color=cmap_1(0.75), label=c[3])
   veryblue_patch = mpatches.Patch(color=cmap_1(1.0), label=c[4])
   space_patch = mpatches.Patch(color='none',label='')
   
   layer["quantile_range"] = pd.qcut(layer[visualize_column],5 )
   a = pd.qcut(layer[visualize_column], 5).cat.categories.right
   b = pd.qcut(layer[visualize_column], 5).cat.categories.left
   c=[str(int(m))+ "-" +str(int(n)) for m,n in zip(b ,a)]

   cmap = cm.get_cmap(cmap)
   title_patch_1 = mpatches.Patch(color='none',label=f'{legend_label}')
   white_patch_1 = mpatches.Patch(color=cmap(0.0), label=c[0])
   lowblue_patch_1 = mpatches.Patch(color=cmap(0.25), label=c[1])
   midblue_patch_1 = mpatches.Patch(color=cmap(0.5), label=c[2])
   highblue_patch_1 = mpatches.Patch(color=cmap(0.75), label=c[3])
   veryblue_patch_1 = mpatches.Patch(color=cmap(1.0), label=c[4])

   ax2 = fig.add_axes(rect=[.73, 0.02,0.22, .88]) #frame_on=False #dimensions [left, bottom, width, height] of the new Axes
   ax2.legend(handles=[ title_patch, white_patch, lowblue_patch, midblue_patch, highblue_patch, veryblue_patch,
                     space_patch, title_patch_1,white_patch_1,lowblue_patch_1, midblue_patch_1, highblue_patch_1, veryblue_patch_1
                     ], loc='upper right', bbox_to_anchor=(1, 1), fontsize='small',title=f'{legend_title}',labelspacing=0.5, handleheight=2, handlelength=3)

   plt.xticks([])
   plt.yticks([])
   # ax.title.set_text(f'{title}')
   ax.set_title(title, fontsize=10, fontweight='bold', wrap=True)
   # ax.axis('tight') #zooms in the to the largest features bounds
   ax.axis('off')
   ax2.axis('off')
   plt.tight_layout()
   ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)


In [23]:
def export_fathom_on_fathom_plus_infrastructure(vector_file, legend_title, crs ,  visualize_column ):
   #  fig = plt.figure(figsize=(width, height))
   height=7
   width=8
   fig, ax = plt.subplots(figsize=(width, height))
   # create flood list
   flood_layers_dict = defaultdict(list)
   file_list = glob.glob(os.path.join(shapefiles, "polygonized_clipped_fathom_1*.shp"))
   for file_path in file_list:
      file_name= os.path.basename(file_path)
      file_name = os.path.splitext(file_name)[0]
      # print(file_name)
      file_name_clean=file_name.replace("-", " ")
      file_name_clean=file_name_clean.split(" ")[3:-1]
      return_period=file_name_clean[0]
      return_period=return_period.split("in")[-1]
      flood_type= file_name_clean[1].title()
      var= file_name_clean[2]
      Year= file_name_clean[4]
      scenario=file_name_clean[5]
      if "SSP" in scenario:
         ssp=scenario
         ssp=ssp.replace("_", "-")
      else:
         ssp="Present"
      # flood_layers_dict[Year].append(file_name)
      flood_layers_dict[Year].append(file_path)
      
   res = [[i for i in flood_layers_dict[x]] for x in flood_layers_dict.keys()] 
   # cmap_list=list(colormaps)
   cmap_list=['RdPu','afmhot_r','Blues']
   for i in res:
      print( i)
      handles_list=[]
      for cmap, j in enumerate(i):
         # for file_path in file_list:
         file_name= os.path.basename(j)
         file_name = os.path.splitext(file_name)[0]
         file_name_clean=file_name.replace("-", " ")
         file_name_clean=file_name_clean.split(" ")[3:-1]
         return_period=file_name_clean[0]
         return_period=return_period.split("in")[-1]
         flood_type= file_name_clean[1].title()
         var= file_name_clean[2]
         Year= file_name_clean[4]
         scenario=file_name_clean[5]
         if "SSP" in scenario:
               ssp=scenario
               ssp=ssp.replace("_", "-")
         else:
               ssp="Present"

         print( j, cmap_list[cmap], Year,ssp , flood_type)
         cmap=cmap_list[cmap]
         layer= gpd.read_file(Path(j)).to_crs(WGS84)
         ax = layer.to_crs(crs).plot(ax=ax, 
                                       column= visualize_column, 
                                       alpha=0, 
                                       facecolor='none',
                                       scheme="quantiles", 
                                       cmap=cmap,
                                       label=f"{flood_type}" ,
                                       legend=False 
                                             )
         
         layer_filtered=layer[layer[visualize_column]>0]
         ax = layer_filtered.to_crs(crs).plot(ax=ax, 
                                       column= visualize_column, 
                                       alpha=.6, 
                                    #    hatch="/",
                                       scheme="quantiles", 
                                       cmap=cmap,
                                       label=f"{flood_type}" ,
                                       legend=False 
                                             )
      
         layer["quantile_range"] = pd.qcut(layer[visualize_column],5 )
         a = pd.qcut(layer[visualize_column], 5).cat.categories.right
         b = pd.qcut(layer[visualize_column], 5).cat.categories.left
         c=[str(int(m))+ "-" +str(int(n)) for m,n in zip(b ,a)]
         
         

         legend_label=f"{flood_type} flood(cm)"  #f"Flood"
         legend_label_patch = Line2D([0], [0], marker='o', color='none', label=f'{legend_label}', markerfacecolor='none', markersize=1)
         handles_list.append(legend_label_patch)
         cmap = cm.get_cmap(cmap)
         for num, increment in enumerate(np.arange(0, 1.25, 0.25)):
            print(num, increment)
            patch = mpatches.Patch(color=cmap(increment), label=c[num])
            handles_list.append(patch)
      
      ax=vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)

      cx.add_basemap(ax, 
                  source=cx.providers.Esri.WorldImagery,  
                  crs=crs,
                  attribution=False, #No citation
                  ) 

      # -----
      index=0
      polygon = vector_file.reset_index().iloc[index]['geometry']
      tags={'amenity':['college', 'dancing_school', 'driving_school', 'kindergarten', \
                     'language_school', 'library', 'surf_school', 'toy_library', 'research_institute', \
                     'training', 'music_school', 'school', 'university'],
                     'landuse': 'education'
                     }
      education=ox.features.features_from_polygon(polygon, tags)
      # tags={'leisure':['park', 'nature_reserve', 'protected_area', 'garden']}
      tags={'leisure':['park',  'garden']}
      park=ox.features.features_from_polygon(polygon, tags)
      park.geometry = park['geometry'].centroid
      tags={'landuse':['commercial', 'industrial', 'construction', 'retail']                   }
      industry=ox.features.features_from_polygon(polygon, tags)
      industry.geometry = industry['geometry'].centroid
      tags={'building': 'government'}
      # government_building=ox.features.features_from_polygon(polygon, tags)
      # government_building.geometry = government_building['geometry'].centroid
      tags = {"building": True}
      building_footprint=ox.features.features_from_polygon(polygon, tags)
      building_footprint.geometry = building_footprint['geometry'].centroid
      tags = {'amenity':['clinic', 'doctors', 'dentist', 'health_post', 'hospital', 'nursing_home', 'pharmacy', 'veterinary']}
      health_care=ox.features.features_from_polygon(polygon, tags)
      health_care.geometry = health_care['geometry'].centroid
      G = ox.graph.graph_from_polygon(polygon, network_type='drive')
      # add travel time based on maximum speed
      G = ox.add_edge_speeds(G) 
      G = ox.add_edge_travel_times(G) 
      G = ox.projection.project_graph(G, to_crs=3857)
      # get edges as Geo data frame
      _, gdf_edges = ox.graph_to_gdfs(G)

      ax = education.to_crs(crs).plot(ax=ax,
                                       alpha=0.6,  
                                       facecolor='olive',
                                       edgecolor='olive' , 
                                       label="Education",
                                       linewidth=3,
                                       markersize=10,
                                       marker="P"  
                                       )
      
      ax = industry.to_crs(crs).plot(ax=ax,
                                       alpha=0.6,  
                                       facecolor='cyan', 
                                       edgecolor='cyan' , 
                                       label="Industries",
                                       linewidth=3,
                                       markersize=10,
                                       marker="p"  
                                       )
      
      ax = park.to_crs(crs).plot(ax=ax,
                                       alpha=0.6,  
                                       facecolor='peru',
                                       edgecolor='peru' , 
                                       label="Parks",
                                       linewidth=3,
                                       markersize=10,
                                       marker="*"   
                                       )
      ax = health_care.to_crs(crs).plot(ax=ax,
                                       alpha=0.6,  
                                       facecolor='red',
                                       edgecolor='red' , 
                                       label="Healthcare",
                                       linewidth=3,
                                       markersize=10,
                                       marker="o"   
                                       )

      ax = gdf_edges.to_crs(crs).plot(ax=ax,
                                       alpha=0.4,  
                                       facecolor='none',
                                       edgecolor='black' , 
                                       label="Roads",
                                       linewidth=.3,
                                       linestyle='dashed',
                                       # zorder=1
                                       )
      lines = [Line2D([0], [0], linestyle="none",  fillstyle=None, marker="P", markersize=10, color='olive' , label='Education'),
               Line2D([0], [0], linestyle="none",  fillstyle=None, marker="p", markersize=10, color='cyan' , label='Industries'),
               Line2D([0], [0], linestyle="none",  fillstyle=None, marker="*", markersize=10, color='peru' , label='Parks'),
               Line2D([0], [0], linestyle="none",  fillstyle=None, marker="o", markersize=10, color='red' , label='Healthcare'),
               Line2D([0], [0], linestyle="dashed",  fillstyle=None, marker="_", markersize=10, color='black' , label='Roads')]
      
      space_patch = mpatches.Patch(color='none',label='')
      handles_list.append(space_patch)
      legend_label_infra=f"Infrastucture"  #f"Flood"
      legend_label_patch = Line2D([0], [0], marker='o', color='none', label=f'{legend_label_infra}', markerfacecolor='none', markersize=1)
      handles_list.append(legend_label_patch)
      
      for l in lines:
         handles_list.append(l)
      ax2 = fig.add_axes(rect=[.72, 0.025,0.22, .88]) #frame_on=False #dimensions [left, bottom, width, height] of the new Axes
      ax2.legend(handles=handles_list, loc='upper right', bbox_to_anchor=(1, 1), fontsize='small',title=f'{legend_title}',labelspacing=0.5, handleheight=2, handlelength=3)


      #------
      plt.xticks([])
      plt.yticks([])
      # ax.title.set_text(f'{title}')
      title= f"Overlay of flood in {Year} with a \n return period of {return_period} years ({ssp})"
      map_output = f"{maps}/{city}_{Year}_{ssp}_{flood_type}_fathom_on_fathom_plus_infrastructure.png"

      ax.set_title(title, fontsize=10, fontweight='bold', wrap=True)
      # ax.axis('tight') #zooms in the to the largest features bounds
      ax.axis('off')
      ax2.axis('off')
      plt.tight_layout()
      ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)


In [24]:
def export_fathom_on_fathom_overlays(vector_file, legend_title, crs ,  visualize_column ):
   #  fig = plt.figure(figsize=(width, height))
   height=7
   width=8
   fig, ax = plt.subplots(figsize=(width, height))
   # create flood list
   flood_layers_dict = defaultdict(list)
   file_list = glob.glob(os.path.join(shapefiles, "polygonized_clipped_fathom_1*.shp"))
   for file_path in file_list:
      file_name= os.path.basename(file_path)
      file_name = os.path.splitext(file_name)[0]
      # print(file_name)
      file_name_clean=file_name.replace("-", " ")
      file_name_clean=file_name_clean.split(" ")[3:-1]
      return_period=file_name_clean[0]
      return_period=return_period.split("in")[-1]
      flood_type= file_name_clean[1].title()
      var= file_name_clean[2]
      Year= file_name_clean[4]
      scenario=file_name_clean[5]
      if "SSP" in scenario:
         ssp=scenario
         ssp=ssp.replace("_", "-")
      else:
         ssp="Present"
      # flood_layers_dict[Year].append(file_name)
      flood_layers_dict[Year].append(file_path)
      
   res = [[i for i in flood_layers_dict[x]] for x in flood_layers_dict.keys()] 
   # cmap_list=list(colormaps)
   cmap_list=['RdPu','afmhot_r','Blues']
   for i in res:
      print( i)
      handles_list=[]
      for cmap, j in enumerate(i):
         # for file_path in file_list:
         file_name= os.path.basename(j)
         file_name = os.path.splitext(file_name)[0]
         file_name_clean=file_name.replace("-", " ")
         file_name_clean=file_name_clean.split(" ")[3:-1]
         return_period=file_name_clean[0]
         return_period=return_period.split("in")[-1]
         flood_type= file_name_clean[1].title()
         var= file_name_clean[2]
         Year= file_name_clean[4]
         scenario=file_name_clean[5]
         if "SSP" in scenario:
               ssp=scenario
               ssp=ssp.replace("_", "-")
         else:
               ssp="Present"

         print( j, cmap_list[cmap], Year,ssp , flood_type)
         cmap=cmap_list[cmap]
         layer= gpd.read_file(Path(j)).to_crs(WGS84)
         ax = layer.to_crs(crs).plot(ax=ax, 
                                       column= visualize_column, 
                                       alpha=0, 
                                       facecolor='none',
                                       scheme="quantiles", 
                                       cmap=cmap,
                                       label=f"{flood_type}" ,
                                       legend=False 
                                             )
         
         layer_filtered=layer[layer[visualize_column]>0]
         ax = layer_filtered.to_crs(crs).plot(ax=ax, 
                                       column= visualize_column, 
                                       alpha=.6, 
                                    #    hatch="/",
                                       scheme="quantiles", 
                                       cmap=cmap,
                                       label=f"{flood_type}" ,
                                       legend=False 
                                             )
      
         layer["quantile_range"] = pd.qcut(layer[visualize_column],5 )
         a = pd.qcut(layer[visualize_column], 5).cat.categories.right
         b = pd.qcut(layer[visualize_column], 5).cat.categories.left
         c=[str(int(m))+ "-" +str(int(n)) for m,n in zip(b ,a)]
         
         

         legend_label=f"{flood_type} flood(cm)"  #f"Flood"
         legend_label_patch = Line2D([0], [0], marker='o', color='none', label=f'{legend_label}', markerfacecolor='none', markersize=1)
         handles_list.append(legend_label_patch)
         cmap = cm.get_cmap(cmap)
         for num, increment in enumerate(np.arange(0, 1.25, 0.25)):
            print(num, increment)
            patch = mpatches.Patch(color=cmap(increment), label=c[num])
            handles_list.append(patch)
       
         ax=vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)

         cx.add_basemap(ax, 
                     source=cx.providers.Esri.WorldImagery,  
                     crs=crs,
                     attribution=False, #No citation
                     ) 

         
         ax2 = fig.add_axes(rect=[.72, 0.025,0.22, .88]) #frame_on=False #dimensions [left, bottom, width, height] of the new Axes
         ax2.legend(handles=handles_list, loc='upper right', bbox_to_anchor=(1, 1), fontsize='small',title=f'{legend_title}',labelspacing=0.5, handleheight=2, handlelength=3)


         #------
         plt.xticks([])
         plt.yticks([])
         # ax.title.set_text(f'{title}')
         title= f"Overlay of flood in {Year} with a \n return period of {return_period} years ({ssp})"
         map_output = f"{maps}/{city}_{Year}_{ssp}_{flood_type}_fathom_on_fathom_overlays.png"

         ax.set_title(title, fontsize=10, fontweight='bold', wrap=True)
         # ax.axis('tight') #zooms in the to the largest features bounds
         ax.axis('off')
         ax2.axis('off')
         plt.tight_layout()
         ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)


In [25]:
stat= [ 'mean','percentile_95']
section= ['fathom']

create_mosaic=create_fathom_mosaic(section=section)
input_dir=PureWindowsPath(os.path.join(data, f"{section[0]}/clean"))
clipped_output_shapefile , df_tranformed =export_fathom(input_dir, section,shp, stat)

titles= ["(Average)", "(95 Percentile)"]
for s, t in zip(stat,titles):  
    fathom_barchart(df=df_tranformed, subtitle= t, y_var=s)


C:\Users\Aziz\Dropbox\CRP\FCS\data\fathom\COASTAL_DEFENDED


RasterioIOError: C:/Users/Aziz/Dropbox/CRP/FCS/data/fathom/COASTAL_DEFENDED/2020: Permission denied

In [ ]:
legends_format= '{:,.0f}'
legends_format_1= '{:,.0f}'
legend_title="Legend"
visualize_column="fathom"
crs = 4326
export_fathom_on_fathom_overlays(vector_file, legend_title, crs ,  visualize_column )
export_fathom_on_fathom_plus_infrastructure(vector_file, legend_title, crs ,  visualize_column )


: 

In [ ]:
def upsample_rasters_to_fathoms_resolution(rasters):
    for root, dirs_list, files_list in os.walk( rasters):
        for file_name in files_list:
            extension = '.tif'
            if os.path.splitext(file_name)[-1] == extension and "clipped_fathom_GLOBAL-1ARCSEC-NW_OFFSET-1in10-PLUVIAL-DEFENDED-DEPTH-2050-SSP2_4.5-PERCENTILE50-v3.0" in file_name:
                # if "SSP1_2020" in file_name:
                file_name_path = os.path.join(root, file_name)
                file_name = os.path.splitext(file_name)[0]
                Year = file_name.split('_')[4].upper() 
                ssp = file_name.split('_')[3].upper() 
                print(file_name, ssp, Year, file_name_path)
                # Get the projection and resolution of fathom and apply it to popdynamic 
                ds=gdal.Open(file_name_path)
                prj=ds.GetProjection()
                print(prj)
                ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
                print("ulx, xres, xskew, uly, yskew, yres", ulx, xres, xskew, uly, yskew, yres)

            if os.path.splitext(file_name)[-1] == extension and "projected_clipped_popdynamics" in file_name:
                if "SSP1_2020" in file_name:
                    file_name_path = os.path.join(root, file_name)
                    file_name = os.path.splitext(file_name)[0]
                    Year = file_name.split('_')[4].upper() 
                    ssp = file_name.split('_')[3].upper() 
                    print(file_name, ssp, Year, file_name_path)
                    
                    # prj="EPSG:4326"
                    upsampled=f'{rasters}/upsample_popdynamics_{ssp}_{Year}.tif'
                    reproj = gdal.Warp(upsampled, file_name_path, dstSRS=prj , xRes = xres, yRes =yres, resampleAlg='near')
                    reproj = None

                    fig, ax = plt.subplots(nrows=1,ncols=2, figsize=(10,10))
                    b1 = rasterio.open(file_name_path)
                    b2 = rasterio.open(upsampled)
                    p1  = rasterio.plot.show(b1, ax=ax[0], cmap='nipy_spectral', title='file_name_path 1')
                    p2  = rasterio.plot.show(b2, ax=ax[1], cmap='nipy_spectral', title='upsampled 2')
                    plt.show()

    return upsampled

upsampled=upsample_rasters_to_fathoms_resolution(rasters)
upsampled


: 

In [ ]:
# from shapely.geometry import box
# import rasterio as rio
# from rasterio.enums import Resampling
# # def upsample_rasters_to_fathoms_resolution(rasters):
# for root, dirs_list, files_list in os.walk( rasters):
#     for file_name in files_list:
#         extension = '.tif'
#         if os.path.splitext(file_name)[-1] == extension and "clipped_fathom_GLOBAL-1ARCSEC-NW_OFFSET-1in10-PLUVIAL-DEFENDED-DEPTH-2050-SSP2_4.5-PERCENTILE50-v3.0" in file_name:
#             # if "SSP1_2020" in file_name:
#             file_name_path = os.path.join(root, file_name)
#             ds=gdal.Open(file_name_path)
#             prj=ds.GetProjection()
#             print(prj)

#         if os.path.splitext(file_name)[-1] == extension and "projected_clipped_popdynamics" in file_name:
#             if "SSP1_2020" in file_name:
#                 file_name_path_1 = os.path.join(root, file_name)
                

# print(file_name_path, file_name_path_1)           

# with rio.open(file_name_path) as ras1, rio.open(file_name_path_1) as ras2:
#     x_scale = ras2.transform.a / ras1.transform.a
#     y_scale = ras2.transform.e / ras1.transform.e

#     # scale image transform
#     transform = ras2.transform * ras2.transform.scale(
#         (ras2.width / ras2.shape[-1]),
#         (ras2.height / ras2.shape[-2])
#     )

#     ext1 = box(*ras1.bounds)
#     ext2 = box(*ras2.bounds)
#     intersection = ext1.intersection(ext2)
#     win1 = rio.windows.from_bounds(*intersection.bounds, ras1.transform)
#     win2 = rio.windows.from_bounds(*intersection.bounds, transform)

#     overlap_1 = ras1.read(window=win1)
#     overlap_2 = ras2.read(
#         window=win2,
#         out_shape=(
#             ras2.count,
#             int(win2.height * y_scale),
#             int(win2.width * x_scale)
#         ),
#         resampling=Resampling.bilinear
#     )

#     print(overlap_1)
#     print(overlap_2)

# new_dataset = rasterio.open(f'{rasters}/test1_.tif', 'w', driver='GTiff',
#                             height = overlap_2.shape[0], width = overlap_2.shape[1],
#                             count=1, dtype=str(overlap_2.dtype),
#                             crs=prj,
#                             transform=transform)

# # new_dataset.write(arr, 1)
# new_dataset.write(overlap_2)
# new_dataset.close()


# from osgeo import gdal

# driver = gdal.GetDriverByName('GTiff')
# file = gdal.Open('/home/user/workspace/raster.tif')
# band = file.GetRasterBand(1)
# lista = band.ReadAsArray()

# # reclassification
# for j in  range(file.RasterXSize):
#     for i in  range(file.RasterYSize):
#         if lista[i,j] < 200:
#             lista[i,j] = 1
#         elif 200 < lista[i,j] < 400:
#             lista[i,j] = 2
#         elif 400 < lista[i,j] < 600:
#             lista[i,j] = 3
#         elif 600 < lista[i,j] < 800:
#             lista[i,j] = 4
#         else:
#             lista[i,j] = 5

# # create new file
# file2 = driver.Create( 'raster2.tif', file.RasterXSize , file.RasterYSize , 1)
# file2.GetRasterBand(1).WriteArray(lista)

# # spatial ref system
# proj = file.GetProjection()
# georef = file.GetGeoTransform()
# file2.SetProjection(proj)
# file2.SetGeoTransform(georef)
# file2.FlushCache()


: 

In [ ]:
# get fathom shps list 
file_list = glob.glob(os.path.join(shapefiles, "polygonized_clipped_fathom_GLOBAL*v3.0.shp"))
files = []
for file_path in file_list:
    file_name= os.path.basename(file_path)
    file_name = os.path.splitext(file_name)[0]
    print(file_name)
    file_name_clean=file_name.replace("-", " ")
    file_name_clean=file_name_clean.split(" ")[3:-1]
    return_period=file_name_clean[0]
    return_period=return_period.split("in")[-1]
    flood_type= file_name_clean[1].title()
    var= file_name_clean[2]
    Year= file_name_clean[4]
    scenario=file_name_clean[5]
    if "SSP" in scenario:
        ssp=scenario
        ssp=ssp.replace("_", "-")
    else:
        ssp="Present"

    #get pop shps list
    file_list_1 = glob.glob(os.path.join(shapefiles, "polygonized_clipped_popdynamics_*.shp"))
    for file_path_1 in file_list_1:
        file_name_1= os.path.basename(file_path_1)
        file_name_1 = os.path.splitext(file_name_1)[0]
        year_1=file_name_1.split("_")[-1]
        ssp_1=file_name_1.split("_")[-2]
        print(file_name_1, ssp_1, year_1)

        layer= gpd.read_file(Path(file_path)).to_crs(WGS84)
        layer_1= gpd.read_file(Path(file_path_1)).to_crs(WGS84)

        legends_format= '{:,.0f}'
        legends_format_1= '{:,.0f}'
        legend_title="Legend"
        legend_label=f"{flood_type} flood(cm)"  #f"Flood"
        legend_label_1=f"Population(x1000)"
        cmap ='afmhot_r'
        cmap_1 ="OrRd"
        visualize_column="fathom"
        visualize_column_1="popdynami"
        crs = 4326
        title= f"Population in {year_1} ({ssp_1}) overlayed on \n {flood_type} flood in {Year} with a \n return period of {return_period} years ({ssp})"
        layer_1[visualize_column_1]=layer_1[visualize_column_1].div(1000).round(0)
        map_output = f"{maps}/{city}_{year_1}_{ssp_1}_pop_{flood_type}_{Year}_fathom_overlays.png"
        export_map_fath=export_map_fathom_overlays(vector_file, 
                                                layer_1,
                                                layer, 
                                                legend_title,
                                                legends_format,
                                                legends_format_1,
                                                legend_label,
                                                legend_label_1, 
                                                crs ,
                                                cmap,
                                                cmap_1,  
                                                visualize_column,
                                                visualize_column_1, 
                                                title, map_output)


: 

Urban land cover overlayed on floods. infras

In [ ]:
# get fathom shps list 
file_list = glob.glob(os.path.join(shapefiles, "polygonized_clipped_fathom_GLOBAL*v3.0.shp"))
for file_path in file_list:
    file_name= os.path.basename(file_path)
    file_name = os.path.splitext(file_name)[0]
    print(file_name)
    file_name_clean=file_name.replace("-", " ")
    file_name_clean=file_name_clean.split(" ")[3:-1]
    return_period=file_name_clean[0]
    return_period=return_period.split("in")[-1]
    flood_type= file_name_clean[1].title()
    var= file_name_clean[2]
    Year= file_name_clean[4]
    scenario=file_name_clean[5]
    if "SSP" in scenario:
        ssp=scenario
        ssp=ssp.replace("_", "-")
    else:
        ssp="Present"

    #get pop shps list
    file_list_1 = glob.glob(os.path.join(shapefiles, "polygonized_clipped_urbanland_*.shp"))
    for file_path_1 in file_list_1:
        file_name_1= os.path.basename(file_path_1)
        file_name_1 = os.path.splitext(file_name_1)[0]
        year_1=file_name_1.split("_")[-2].upper()
        year_1=year_1.split("GDP")[-1].upper()
        ssp_1=file_name_1.split("_")[-1].upper() 
        print(file_name_1, ssp_1, year_1)
        layer= gpd.read_file(Path(file_path)).to_crs(WGS84)
        layer_1= gpd.read_file(Path(file_path_1)).to_crs(WGS84)
        legends_format= '{:,.0f}'
        legends_format_1= '{:,.0f}'
        legend_title="Legend"
        legend_label=f"{flood_type} flood(cm)"  #f"Flood"
        legend_label_1="% Urban Land"
        
        cmap ='afmhot_r'
        cmap_1 ="RdPu"
        visualize_column="fathom"
        visualize_column_1="urbanland"
        crs = 4326
        title= f"% Urban in {year_1} ({ssp_1}) overlayed on \n {flood_type} flood in {Year} with a \n return period of {return_period} years ({ssp})"
        layer_1[visualize_column_1]=layer_1[visualize_column_1].multiply(100).round(0)

        map_output = f"{maps}/{city}_{year_1}_{ssp_1}_urbanland_{flood_type}_{Year}_fathom_overlays.png"
        export_map_fath=export_map_fathom_overlays(vector_file, 
                                                layer_1,
                                                layer, 
                                                legend_title,
                                                legends_format,
                                                legends_format_1,
                                                legend_label,
                                                legend_label_1, 
                                                crs ,
                                                cmap,cmap_1,  
                                                visualize_column,
                                                visualize_column_1, 
                                                title, map_output)


: 

In [ ]:
# get fathom shps list 
# flood_layers=[]
# years=[]

flood_layers_dict = defaultdict(list)
file_list = glob.glob(os.path.join(shapefiles, "polygonized_clipped_fathom_GLOBAL*v3.0.shp"))
for file_path in file_list:
    file_name= os.path.basename(file_path)
    file_name = os.path.splitext(file_name)[0]
    # print(file_name)
    file_name_clean=file_name.replace("-", " ")
    file_name_clean=file_name_clean.split(" ")[3:-1]
    return_period=file_name_clean[0]
    return_period=return_period.split("in")[-1]
    flood_type= file_name_clean[1].title()
    var= file_name_clean[2]
    Year= file_name_clean[4]
    scenario=file_name_clean[5]
    if "SSP" in scenario:
        ssp=scenario
        ssp=ssp.replace("_", "-")
    else:
        ssp="Present"
    # flood_layers_dict[Year].append(file_name)
    flood_layers_dict[Year].append(file_path)

res = [[i for i in flood_layers_dict[x]] for x in flood_layers_dict.keys()] 
# cmap_list=["blue", "red", "cyan"]
cmap_list=list(colormaps)
for i in res:
    print( i)
    for cmap, j in enumerate(i):
        # for file_path in file_list:
        file_name= os.path.basename(j)
        file_name = os.path.splitext(file_name)[0]
        file_name_clean=file_name.replace("-", " ")
        file_name_clean=file_name_clean.split(" ")[3:-1]
        return_period=file_name_clean[0]
        return_period=return_period.split("in")[-1]
        flood_type= file_name_clean[1].title()
        var= file_name_clean[2]
        Year= file_name_clean[4]
        scenario=file_name_clean[5]
        if "SSP" in scenario:
            ssp=scenario
            ssp=ssp.replace("_", "-")
        else:
            ssp="Present"
        print( j, cmap_list[cmap], Year,ssp , flood_type)

         

: 

In [ ]:
There's another way to do this, also. You can load in your DEM as a numpy array using

dem_obj = gdal.Open(path_to_DEM)
I = dem_obj.ReadAsArray()
Now set all the values equal to nan:

I = I * np.nan
Now you need to load your rivers raster:

rr_obj = gdal.Open(path_to_riv_raster)
Ir = rr_obj.ReadAsArray()
I would resample the river raster next, using skimage.transform's resize function. You need to know the size of your resampled river raster, though, which you can obtain through the geotransforms of both:

gt_dem = dem_obj.GetGeoTransform()
new_rr_shape = (Ir.shape[0]/gt_dem[4], Ir.shape[1]/gt_dem[1])
Now resize the river raster so that the pixels are the same size as the DEM:

import skimage.transform as st 
Ir = st.resize(Ir, new_rr_shape, mode='constant')
Now you have your river raster at the same resolution as your DEM. The next step is to place it in the correct position within the np.nan array we created from the DEM. To do that, you need to use the geotransforms of both images to align the top-left-most pixel.

gt_rr = rr_obj.GetGeoTransform()
ncols_offset = (gt_dem[0] - gt_rr[0]) / gt_dem[1]
nrows_offset = (gt_dem[3] - gt_rr[3]) / gt_dem[4]
You'll notice that your ncols and nrows _offsets are not integers, but you'll need them to be. You can use either round() or int() (or other choices) to make them integers; the choice you make will shift the river raster no more than a pixel in either direction.

Now that you have your river raster resampled to the same resolution, and an empty (nan-filled, really) DEM template raster to put them into, and the row, col offsets, you can just put the river raster in with numpy:

import numpy as np
I[nrows_offset:nrows_offset+Ir.shape[1], ncols_offset:ncols_offset+Ir.shape[0]] = Ir
Finally, you'll want to save your resampled, rescaled river raster as a geotiff. I will post code here you can use as a template:

def write_geotiff(raster, gt, wkt, outputpath, dtype=gdal.GDT_UInt16, options=['COMPRESS=LZW'], color_table=0, nbands=1, nodata=False):

    width = np.shape(raster)[1]
    height = np.shape(raster)[0]

    # Prepare destination file
    driver = gdal.GetDriverByName("GTiff")
    if options != 0:
        dest = driver.Create(outputpath, width, height, nbands, dtype, options)
    else:
        dest = driver.Create(outputpath, width, height, nbands, dtype)

    # Write output raster
    if color_table != 0:
        dest.GetRasterBand(1).SetColorTable(color_table)

    dest.GetRasterBand(1).WriteArray(raster)

    if nodata is not False:
        dest.GetRasterBand(1).SetNoDataValue(nodata)

    # Set transform and projection
    dest.SetGeoTransform(gt)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(wkt)
    dest.SetProjection(srs.ExportToWkt())

    # Close output raster dataset 
    dest = None
Your call should look something like:

write_geotiff(I, gt_dem, dem_obj.GetProjection(), path_to_output_file, nodata=np.nan)



: 

In [ ]:
gdal.Warp('outputRaster.tif', 'inputRaster.tif', xRes=5, yRes=5)


: 

: 

In [ ]:
ample of upsampling by a factor of 2 using the bilinear resampling method.

import rasterio
from rasterio.enums import Resampling

upscale_factor = 2

with rasterio.open("example.tif") as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * upscale_factor),
            int(dataset.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

: 

Overlay all floods ontop of each other. Present and 2050 on respective ones

In [ ]:
# # africa.loc[africa['EAC'] == 1].plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

# # def export_fathom():

# section= ["fathom"]
# stat= ['mean']
# # cmap= "magma" #"Blues"
# cmap='Reds' 
# legends_format= '{:.2f}'

# # export_fathom()
# hexbins_projected= create_hexbins(shapefiles, shp=shp, hexbin_res=hexbin_res, WGS84=WGS84)
# vector_file = gpd.read_file(shp).to_crs(WGS84)
# for subdir in section:
#     combined_results = []
#     extension = '.tif'
#     for root, dirs_list, files_list in os.walk( os.path.join(data, subdir)):
#         for file_name in files_list:
#             if os.path.splitext(file_name)[-1] == extension:
#                 file_name_path = os.path.join(root, file_name)
#                 file_name = os.path.splitext(file_name)[0]
#                 flood_type=file_name_path.split('\\')[-2].title() 
#                 Year = file_name.split('in')[1].title() 
#                 ssp= flood_type
#                 print(f"file_name: {file_name} ---  Year:{Year} --- flood_type:{flood_type}")
#                 file_name_projected= f'{file_name}_wsg84.tif'
#                 file_name_projected= file_name_projected.replace("-", "_")
#                 output_raster=os.path.join(root, file_name_projected) 
#                 rst = file_name_path 
#                 raster = rasterio.open(rst)
#                 # vector = load_shapefile(shp)
#                 vector = load_shapefile_buffer(shp)
#                 fathom_clean= os.path.join(data,  "fathomclean")
#                 fathom_clean=create_dir(fathom_clean)
#                 output_fathom=create_dir(os.path.join(fathom_clean, flood_type)) 
#                 clean_rst = f'{output_fathom}/{file_name}.tif'
#                 clip_and_export_raster_fathom(input_gpdf=vector, raster=rst , out_raster=clean_rst)
#                 rst=clean_rst
#                 raster = rasterio.open(rst)
#                 p_vector = reproject_gpdf(vector, raster)
#                 stats_to_get = list_statistics(stat)
#                 df = calculate_zonal_stats(p_vector, rst, stats_to_get)
#                 df['Scenario'] = flood_type
#                 df['Year'] = Year
#                 # print(file_name,  file_name_path, subdir, ssp, Year)
#                 combined_results.append(df)
#                 #clip and export rasters for the pertinent ssp and year combo
#                 if (flood_type!="" ) and (Year=="10" ):
#                     out_rst = f'{rasters}/clipped_{subdir}_{file_name}.tif'
#                     # clip_and_export_raster(input_gpdf=vector, raster=rst , out_raster=out_rst)
#                     out_rst_2 = f'{rasters}/clipped_2_{subdir}_{file_name}.tif'
#                     map_title= f"{flood_type} flood event in {Year} years"
#                     legend_title=f"{flood_type} flood"
#                     [statistic]= stats_to_get
#                     hexbins_projected , max , min= hexbin_zonal_stats(shp=shp, \
#                         hexbins_projected=hexbins_projected , crs=WGS84 , \
#                             raster=rst , out_raster=out_rst_2, statistic=statistic)
#                     crs = 4326
#                     projected_raster = f'{rasters}/projected_clipped_{subdir}_{file_name}.tif'
#                     unprojected_raster = out_rst_2
#                     reproject_rast= reproject_rasters(crs=crs, unprojected_raster=unprojected_raster, \
#                                                         projected_raster=projected_raster)
#                     # cmap = plt.get_cmap('RdYlGn', 10)
#                     # cmap = truncate_colormap(cmap, 0.1, 1)
#                     map_output = f"{maps}/{city}_{subdir}_{file_name}_cmap_max_div_4.png"
#                     document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
#                     #export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)
#                     output_shapefile = f'{shapefiles}/polygonized_{subdir}_{file_name}.shp'
#                     output_geojson= f'{shapefiles}/geojson_{subdir}_{file_name}.geojson'
#                     output_csv= f'{tables}/csv_{subdir}_{file_name}.csv'
#                     driver= 'ESRI Shapefile'
#                     mask_value=None
#                     raster_val= subdir[0:9]
#                     name = polygonize(raster_file=projected_raster ,crs=EPSG_str, output_shapefile=output_shapefile, \
#                                         driver=driver, mask_value=mask_value, raster_val=raster_val, ssp=ssp, Year=Year)
#                     clipped_output_shapefile = f'{shapefiles}/polygonized_clipped_{subdir}_{file_name}.shp'
#                     clip_raster_shapes= clip_raster_shapefiles(shp=shp, output_shapefile=output_shapefile, crs=crs,\
#                                                                 output_geojson= output_geojson, output_csv=output_csv, \
#                                                                 clipped_output_shapefile=clipped_output_shapefile)                        
#                     polygonized_shapefile = gpd.read_file(clipped_output_shapefile).to_crs(WGS84) 
#                     polygonized_shapefile[raster_val]=polygonized_shapefile[raster_val].round(2)
#                     export_the_map= export_map(map_dir= output, vector_file=vector_file , \
#                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
#                             title=map_title, map_output=map_output)
#                     export_the_osm_map= export_map_osm(map_dir= output, vector_file=vector_file , \
#                             hexagons=polygonized_shapefile,legend_title=legend_title, legends_format=legends_format, crs=crs, cmap=cmap , visualize_column=raster_val,  \
#                             title=map_title, map_output=map_output)


#     # Append vertically and then pivot horizontally.
#     df = gpd.GeoDataFrame( pd.concat( combined_results, ignore_index=True) )
#     # Wide
#     [stats_to_get] = stats_to_get
#     df_tranformed= df.pivot(index=['Scenario'], columns='Year', values=stats_to_get)        
#     # Long
#     df_graph = df_tranformed.melt(ignore_index=False).reset_index()
#     # export csv of wide
#     df_tranformed.to_csv(f"{tables}/{country}_{city}_{subdir}.csv")
#     #run the visualize function to export the heatmap
#     # visualize_gdp_figure(df_tranformed, country, city, subdir)
#     # visualize_gdp_graph(df_graph, country, city, subdir)


: 

use 100 return period. 


Demographic pyramids functions from Bramkas note

In [ ]:
def demographic_projection(city, country_iso3, country, ssp_to_use, section):

    gc_city_folder = os.path.join(data, f'{section[0]}/GC_countries/')
    ssp_master_fn = os.path.join(data, f'{section[0]}/SspDb_country_data_2013-06-12.csv')
    # Step 0.1: Get country-level projections
    ssp_master = pd.read_csv(ssp_master_fn)
    ssp_country = ssp_master.loc[ssp_master['REGION']==country_iso3]
    ssp_country.reset_index(inplace=True, drop=True)
    if len(ssp_country)==0:
        print('Country not found in SSP database. Check if ISO3 name of the country is correct, and whether the country is actually available in the SSP database.')
        return 0,0
    else:
        del ssp_master
        
    # Step 0.2: Get city's historical demographic change
    global_cities = pd.read_csv(gc_city_folder+'GC_{}.csv'.format(country_iso3))
    gc_city = global_cities.loc[global_cities['Location']==city]
    if len(gc_city)==0:
        print('City not found in the Global Cities database. Manually check if the city name spelling is correct, and whether the city is actually available in the Global Cities database.')
        return 0,0
    else:
        del global_cities
        
    ######################################################################################################
    ############## STEP 1: Get historical population change at country level
    ######################################################################################################
    all_variables = list(np.unique(ssp_country['VARIABLE']))
    all_age_variables = [x for x in all_variables if 'Aged' in x]

    children_ssp = ['Aged0-4', 'Aged5-9', 'Aged10-14']
    young_adult_ssp = ['Aged15-19', 'Aged20-24', 'Aged25-29', 'Aged30-34']
    adult_ssp = ['Aged35-39', 'Aged40-44', 'Aged45-49', 'Aged50-54', 'Aged55-59', 'Aged60-64']
    elderly_ssp = ['Aged65-69', 'Aged70-74', 'Aged75-79', 'Aged80-84', 'Aged85-89', 'Aged90-94', 'Aged95-99', 'Aged100+']

    buckets_ssp = {'children': children_ssp, 'young_adult': young_adult_ssp, 'adult': adult_ssp, 'elderly': elderly_ssp}
    
    def get_population_ssp_year(ssp_country, ssp, year, buckets_ssp, all_variables):
    
        age_groups = []
        population = []

        for age_group in buckets_ssp.keys():

            # Filter relevant variables for the age group
            filtered_vars = list(filter(lambda x: np.sum([y in x for y in buckets_ssp[age_group]])>0, all_variables))
            filtered_vars = [x for x in filtered_vars if len(x.split("|"))==3]

            # Filter the SSP dataframe
            filtered_dataframe = ssp_country.loc[ssp_country['VARIABLE'].isin(filtered_vars)]
            filtered_dataframe = filtered_dataframe.loc[filtered_dataframe['SCENARIO'].str.contains(ssp)]

            age_groups.append(age_group)
            try:
                population.append(filtered_dataframe[year].sum())
            except:
                population.append(filtered_dataframe[str(year)].sum())

        output = pd.DataFrame()
        output['age_group'] = age_groups
        output['population'] = population
        output['ssp'] = ssp
        output['year'] = year

        return output
    
    pop_2010_sspA = get_population_ssp_year(ssp_country, ssp_to_use, 2010, buckets_ssp, all_variables)
    pop_2020_sspA = get_population_ssp_year(ssp_country, ssp_to_use, 2020, buckets_ssp, all_variables)

    # get historical country change
    hstr_chg_country = {}
    for age_group in buckets_ssp.keys():
        pop_2020 = pop_2020_sspA.loc[pop_2020_sspA['age_group']==age_group, 'population'].values[0]
        pop_2010 = pop_2010_sspA.loc[pop_2010_sspA['age_group']==age_group, 'population'].values[0]
        hstr_chg_country[age_group] =  (pop_2020 - pop_2010) / pop_2010
        
    ######################################################################################################
    ############## STEP 2: Get historical population change at city level
    ######################################################################################################
    gc_city_slc = gc_city.loc[gc_city['Year'].isin([2010, 2020])]

    children_gc = ['POP0_14T']
    young_adult_gc = ['POP15_19T', 'POP20_24T', 'POP25_29T', 'POP30_34T']
    adult_gc = ['POP35_39T', 'POP40_44T', 'POP45_49T', 'POP50_54T', 'POP55_59T', 'POP60_64T']
    elderly_gc = ['POP65_T']

    buckets_gc = {'children': children_gc, 'young_adult': young_adult_gc, 'adult': adult_gc, 'elderly': elderly_gc}

    # get historical city change
    hstr_chg_city = {}
    for age_group in buckets_gc.keys():
        pop_2020 = gc_city_slc.loc[gc_city_slc['Year']==2020][buckets_gc[age_group]].sum().values[0]
        pop_2010 = gc_city_slc.loc[gc_city_slc['Year']==2010][buckets_gc[age_group]].sum().values[0]
        hstr_chg_city[age_group] =  (pop_2020 - pop_2010) / pop_2010
        
    ######################################################################################################
    ############## STEP 3: Calculate scaling factors between country-level and city-level demographic change
    ######################################################################################################
    scaling_factor = {}
    for age_group in buckets_gc.keys():
        scaling_factor[age_group] = hstr_chg_city[age_group] / hstr_chg_country[age_group]
        
    ######################################################################################################
    ############## STEP 4-6: Calculate future city population
    ######################################################################################################
    country_pop = pd.DataFrame(columns=['Year'] + list(buckets_gc.keys()))

    city_pop = pd.DataFrame()
    city_pop['Year'] = [2020]
    for age_group in buckets_gc.keys():
        city_pop[age_group] = [gc_city_slc.loc[gc_city_slc['Year']==2020][buckets_gc[age_group]].sum().values[0]]

    for i in np.arange(2025,2105, 5):
        
        # Step 4: Get future projections at country level
        pop_future1_sspA = get_population_ssp_year(ssp_country, ssp_to_use, i-5, buckets_ssp, all_variables)
        pop_future2_sspA = get_population_ssp_year(ssp_country, ssp_to_use, i, buckets_ssp, all_variables)
        future_chg_country = {}
        for age_group in buckets_ssp.keys():
            pop_future1 = pop_future1_sspA.loc[pop_future1_sspA['age_group']==age_group, 'population'].values[0]
            pop_future2 = pop_future2_sspA.loc[pop_future2_sspA['age_group']==age_group, 'population'].values[0]
            future_chg_country[age_group] =  (pop_future2 - pop_future1) / pop_future1

        # Step 5: Adjust future demographic change at country level to the city level, based on the scaling factors 
        future_chg_city = {}
        for age_group in buckets_ssp.keys():
            future_chg_city[age_group] = future_chg_country[age_group] * scaling_factor[age_group]

        # Step 6: Calculate future population for each demographic group at city level
        to_input = []
        to_input.append(i)
        for age_group in buckets_gc.keys():
            current_pop = city_pop.loc[city_pop['Year']==i-5, age_group].values[0]
            future_pop = current_pop + current_pop * future_chg_city[age_group]
            to_input.append(future_pop)

        city_pop.loc[len(city_pop)] = to_input

        # Save also country-level projection, just in case needed
        if i == 2025:
            country_pop.loc[len(country_pop)] = [i-5] + [pop_future1_sspA.loc[pop_future1_sspA['age_group']==list(buckets_gc.keys())[j]]['population'].values[0] for j in range(len(list(buckets_gc.keys())))]
        country_pop.loc[len(country_pop)] = [i] + [pop_future2_sspA.loc[pop_future2_sspA['age_group']==list(buckets_gc.keys())[j]]['population'].values[0] for j in range(len(list(buckets_gc.keys())))]
        
    city_pop['SSP'] = ssp_to_use
    country_pop['SSP'] = ssp_to_use
        
    return city_pop, country_pop

# Add a plt.figure and then plot the pyramids on the same axis
def pyramid(city_pop, year1, year2, ssp):
    
    # Slice the pyramid
    city_pop_pyramid = city_pop.loc[city_pop['Year'].isin([year1, year2])]
    city_pop_pyramid = city_pop_pyramid.loc[city_pop_pyramid['SSP']==ssp]
    city_pop_pyramid = city_pop_pyramid[city_pop_pyramid.columns[1:-1]]
    city_pop_pyramid = city_pop_pyramid.div(city_pop_pyramid.sum(axis=1), axis=0)
    city_pop_pyramid *= 100
    city_pop_pyramid['Year'] = [year1, year2]
    
    # Visualization
    arr1 = city_pop_pyramid.loc[city_pop_pyramid['Year']==year1].values[0][:-1]
    arr1 *= -1
    arr2 = city_pop_pyramid.loc[city_pop_pyramid['Year']==year2].values[0][:-1]
    
    minval = np.floor(np.min(arr1) / 20)
    maxval = np.ceil(np.max(arr2)  / 20)
    topper = np.max([np.abs(minval), np.abs(maxval)])
    fig_sns, ax_sns = plt.subplots(figsize=(7, 5))

    ax_sns = sns.barplot(x=arr1, y=city_pop_pyramid.columns[:-1], color="blue", ax=ax_sns)
    ax_sns = sns.barplot(x=arr2, y=city_pop_pyramid.columns[:-1], color="green", ax=ax_sns)


    plt.xticks(ticks=[-topper*20, -topper*10, 0, topper*10, topper*20],
    labels=['{}%'.format(topper*20), '{}%'.format(topper*10), '0', '{}%'.format(topper*10), '{}%'.format(topper*20)])

    ax_sns.text(x=0.25, y=1.05, s=year1, transform = ax_sns.transAxes, horizontalalignment='center',
             fontsize=12, color='blue')

    ax_sns.text(x=0.75, y=1.05, s=year2, transform = ax_sns.transAxes, horizontalalignment='center',
             fontsize=12, color='green')

    ax_sns.text(x=0.5, y=1.08, s=ssp, transform = ax_sns.transAxes, horizontalalignment='center',
             fontsize=15, color='black')

    ax_sns.axvline(color='white')

    ax_sns.figure.savefig(f"{maps}/{country}_{city}_{section[0]}_{ssp}_{year1}_{year2}_pyramid.jpeg",
            dpi=300, bbox_inches='tight')


def pop_lineplot(city_pop, ssp):

    city_pop_ = city_pop.loc[city_pop['SSP']==ssp]

    fig, ax = plt.subplots()

    sns.lineplot(x='Year', y='children', data=city_pop_, color='#1b9e77', ax=ax)
    sns.lineplot(x='Year', y='young_adult', data=city_pop_, color='#d95f02', ax=ax)
    sns.lineplot(x='Year', y='adult', data=city_pop_, color='#7570b3', ax=ax)
    sns.lineplot(x='Year', y='elderly', data=city_pop_, color='#e7298a', ax=ax)

    ax.legend(['Children', 'Young adult', 'Adult', 'Elderly'], loc='upper left', bbox_to_anchor=(0.98, 1))

    ax.set_ylabel("Population ('000)")

    ax.set_title("{} - {}".format(city,ssp), fontsize=14)

    sns.despine()
    
    ax.figure.savefig(f"{maps}/{country}_{city}_{section[0]}_{ssp}_lineplot.jpeg",
            dpi=300, bbox_inches='tight')




: 

Analysis

In [ ]:
if __name__ ==  '__main__':    
    country_iso3 = 'BGD'
    section= ['demographic']
    city_pop_ssp2, country_pop_ssp2 = demographic_projection(city=city, 
                                                            country_iso3=country_iso3, 
                                                            country=country, 
                                                            ssp_to_use='SSP2', section=section)

    city_pop_ssp5, country_pop_ssp5 = demographic_projection(city=city, 
                                                            country_iso3=country_iso3, 
                                                            country=country, 
                                                            ssp_to_use='SSP5', section=section)

    city_pop = city_pop_ssp2._append(city_pop_ssp5)
    city_pop = city_pop.loc[city_pop['Year']<=2070]
    city_pop.reset_index(inplace=True, drop=True)

    country_pop = country_pop_ssp2._append(country_pop_ssp5)
    country_pop = country_pop.loc[country_pop['Year']<=2070]
    country_pop.reset_index(inplace=True, drop=True)
    year1_list=[2020, 2050]
    year2_list=[2050, 2070]
    ssp_list=["SSP2", "SSP5"]
    for ssp in ssp_list:
      pop_lineplot(city_pop, ssp)
      for year1, year2 in zip(year1_list, year2_list):
         print(year1, year2, ssp)
         export_pyramid= pyramid(city_pop, year1, year2, ssp)
    

: 

Incorporate the tropical cyclones 

In [ ]:
def get_wind_speed(gdf, country, city):
    # Read data
    # gdf = gpd.read_file(gdf_fn)
    city_centroid = gdf.iloc[0]['geometry'].centroid
    coords = [(city_centroid.x, city_centroid.y)] # get centroid of the city
    coords += [x for x in gdf.iloc[0]['geometry'].exterior.coords] # get exterior of the city
    output = pd.DataFrame()

    # List of return periods and regions
    all_rps = [10, 20, 30, 40, 50, 60, 70, 80, 90, 
               100, 200, 300, 500, 600, 700, 800, 900,
               1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
    all_regions = ['EP', 'NA', 'NI', 'SI', 'SP', 'WP']

    # Get wind speed in current climate
    max_vals = []

    for rp in all_rps:
        vals = []
        for region in all_regions:
            # raster_file = r"inputs/STORM_current/FIXED_RP/STORM_FIXED_RETURN_PERIODS_{}_{}_YR_RP.tif".format(region, rp)
            raster_file = os.path.join(data, f'{section[0]}/STORM_current/FIXED_RP/STORM_FIXED_RETURN_PERIODS_{region}_{rp}_YR_RP.tif')
            src = rasterio.open(raster_file)
            vals.append([x for x in src.sample(coords)])

        max_vals.append(np.max(vals)) # take maximum value across all sampled points

    output['RPs'] = all_rps
    output['Current'] = max_vals

    # List of all climate models used
    all_gcms = ['CMCC-CM2-VHR4',
                'CNRM-CM6-1-HR',
                'EC-Earth3P-HR',
                'HADGEM3-GC31-HM']
    
    # Get wind speed in future climate
    for gcm in all_gcms:
        max_vals = []

        for rp in all_rps:

            vals = []

            for region in all_regions:
                # raster_file = r"inputs/STORM_climate_change/FIXED_RP/{}/STORM_FIXED_RETURN_PERIODS_{}_{}_{}_YR_RP.tif".format(gcm, gcm, region, rp)
                raster_file = os.path.join(data, f'{section[0]}/STORM_climate_change/FIXED_RP/{gcm}/STORM_FIXED_RETURN_PERIODS_{gcm}_{region}_{rp}_YR_RP.tif')
                src = rasterio.open(raster_file)
                vals.append([x for x in src.sample(coords)])
            max_vals.append(np.max(vals)) # take maximum value across all sampled points
        output[gcm] = max_vals
    # Save the output
    output.to_csv(f"{tables}/{country}_{city}_{section[0]}_wind.csv")
    
    return output

def visualize_results(output, country, city):
    
    # List of all climate models used
    all_gcms = ['CMCC-CM2-VHR4',
                'CNRM-CM6-1-HR',
                'EC-Earth3P-HR',
                'HADGEM3-GC31-HM']
    
    # Transform the output into something seaborn-friendly
    output_climate = output[[x for x in output.columns if x!='Current']] 
    output_climate = pd.melt(output_climate, id_vars='RPs', value_vars=all_gcms)
    output_climate.columns = ['RPs', 'GCM', 'Wind speed']
    
    # Get minimum and maximum wind speed for visualization
    minval = np.unique(output[output.columns[1:]])[1]
    maxval = np.unique(output[output.columns[1:]])[-1]
    
    # Main visualization code
    fig, ax = plt.subplots(figsize=(5,5))
    sns.lineplot(data=output_climate, x="RPs", y="Wind speed", label='Climate change\n(2015-2050)', color='indianred', ax=ax)
    ax.plot(output['RPs'], output['Current'], label='Current\n(1980-2017)')
    ax.set_ylim(minval, maxval)
    plt.xscale('log')
    plt.legend(loc='upper left')
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.set_xlabel('Return period', fontsize=12)
    ax.set_ylabel('Maximum wind speed (m/s)', fontsize=12)
    plt.title('Maximum tropical cyclone\nwind speed in {}'.format(city))
    plt.grid()
    
    if maxval > 32:
        ax.text(x= 14000, y=32, s='-- Cat 1')
    if maxval > 42:
        ax.text(x= 14000, y=42, s='-- Cat 2')
    if maxval > 49:
        ax.text(x= 14000, y=49, s='-- Cat 3')
    if maxval > 58:
        ax.text(x= 14000, y=58, s='-- Cat 4')
    if maxval > 70:
        ax.text(x= 14000, y=70, s='-- Cat 5')
        
    # Save and close
    plt.savefig(f"{maps}/{country}_{city}_{section[0]}_cyclone.jpeg", dpi=300, bbox_inches='tight')
    plt.show()

: 

In [ ]:
if __name__ ==  '__main__':    
    section= ['tropicalcyclones']
    gdf =  gpd.read_file(shp).to_crs(WGS84) #r"inputs/Ukhiya.shp"
    output = get_wind_speed(gdf, country, city)
    visualize_results(output, country, city)

: 

Main loop

In [ ]:
%%time
if __name__ ==  '__main__': 
    if socio_econmic:   
        section= ['popdynamics']
        stat= ['sum']
        cmap = "OrRd"
        legends_format= '{:,.0f}'
        export_popdynamics()
        
        section= ['gdp'] 
        stat= ['sum']
        cmap= "RdPu"
        legends_format= '{:,.0f}'
        export_gdp()

        # section= ['urbanland']  
        # stat= ['mean']
        # cmap="YlGnBu"
        # legends_format= '{:,.0f}'
        # export_urbanland()

        # section= ['heatflux']
        # stat= ['mean']
        # # cmap= "magma" #"Blues"
        # cmap='YlOrRd' 
        # legends_format= '{:,.0f}'   
        # export_heatflux()

        # section= ['urbanheatisland']
        # stat= ['mean']
        # cmap='YlOrRd'
        # legends_format= '{:.2f}'
        # export_urbanheatisland()

: 

In [ ]:
%%time
if __name__ ==  '__main__': 
    if socio_econmic:  
        section= ['popdynamics']
        stat= ['sum']
        cmap = "OrRd"
        legends_format= '{:,.0f}'
                
        # ssp_list=["SSP1", "SSP2","SSP3","SSP4","SSP5"]
        ssp_list=["SSP2", "SSP5"]
        start_year_list=["2030","2030"]
        end_year_list=["2050","2100"]

        # for input_ssp in ssp_list:
        #     for start_year, end_year in zip(start_year_list, end_year_list):
        #         print(input_ssp, start_year, end_year)
        #         polygonized_shapefile=export_popdynamics_change(input_ssp, start_year, end_year)

        # section= ['gdp'] 
        # stat= ['sum']
        # cmap= "RdPu"
        # legends_format= '{:,.0f}'

        # for input_ssp in ssp_list:
        #     for start_year, end_year in zip(start_year_list, end_year_list):
        #         print(input_ssp, start_year, end_year)
        #         polygonized_shapefile=export_gdp_change(input_ssp, start_year, end_year)


        # section= ['urbanland']  
        # stat= ['mean']
        # cmap="YlGnBu"
        # legends_format= '{:,.2f}'

        # for input_ssp in ssp_list:
        #     for start_year, end_year in zip(start_year_list, end_year_list):
        #         print(input_ssp, start_year, end_year)
        #         polygonized_shapefile=export_urbanland_change(input_ssp, start_year, end_year)

 
        # do this after fathom
        # section= ['urbanheatisland']
        # stat= ['mean']
        # cmap='YlOrRd'
        # legends_format= '{:.2f}'
        # export_urbanheatisland()

       # ssp_list=["SSP1", "SSP2","SSP3","SSP4","SSP5"]
        ssp_list=["SSP3"] #one only
        start_year_list=["2010"]
        end_year_list=["2050"]

        section= ['heatflux']
        stat= ['mean']
        # cmap= "magma" #"Blues"
        cmap='YlOrRd' 
        legends_format= '{:,.0f}'   
        for input_ssp in ssp_list:
            for start_year, end_year in zip(start_year_list, end_year_list):
                print(input_ssp, start_year, end_year)
                polygonized_shapefile=export_heatflux_change(input_ssp, start_year, end_year) #error


: 

In [ ]:
%%time
if __name__ ==  '__main__':  
    if cckp:     
        section= ['Precipitation']
        appended_data= export_cckp_data(section=section)
        export_graph= export_cckp_graph(df=appended_data , x_var_name='time', y_var_names='Variable', subdir='Precipitation', city=city)   
        
        section= ['Temperature']
        df= export_cckp_data(section=section)
        export_graph= export_cckp_graph(df=df , x_var_name='time', y_var_names='Variable', subdir='Temperature', city=city)   


: 

In [ ]:
%%time
if __name__ ==  '__main__':    
    section= ['globalerosion']
    polygon=get_city_poolygon(shp)
    df=process_erosion()
    map_storms(appended_data=df)

: 












**Process and Map Storms**

In [ ]:
images_displayed=f'{base_dir}/data/images_displayed'  # Input data directory
visualize_image(images_displayed)

: 

In [ ]:
# vector = load_shapefile(shp).to_crs(WGS84)
# vector=vector.reset_index()
# # gdf_city = gpd.read_file(shapefile_path).to_crs(WGS84)

# # OSM admin layers
# tags = {"boundary":"administrative" }
# minx, miny, maxx, maxy = vector.to_crs(WGS84).total_bounds
# all_admin_layers =ox.geometries.geometries_from_bbox(miny,maxy,minx, maxx, tags)
# # Get the lowest 
# lowest_admin_level=all_admin_layers["admin_level"].mode()[0]
# vector_gdf = all_admin_layers[all_admin_layers['admin_level'] == lowest_admin_level]
# lowest_admin_level_name="Settlement"
# vector_gdf["pre_clip_area"] = vector_gdf['geometry'].area
# sub_city_gdf = gpd.clip(vector_gdf.to_crs(crs), vector.to_crs(crs))
# sub_city_gdf["post_clip_area"] = sub_city_gdf['geometry'].area
# sub_city_gdf["pct_clip_area"] = (sub_city_gdf["post_clip_area"]/sub_city_gdf["pre_clip_area"])*100
# sub_city_gdf = sub_city_gdf[sub_city_gdf['pct_clip_area'] > 50]

# # fill english names with native names
# sub_city_gdf['name:en']=sub_city_gdf['name:en'].str.strip().replace('', np.nan).fillna(sub_city_gdf['name'])
 


: 

In [ ]:
def create_city_map(country, city, vector_file,sub_city_gdf,admin_name, lowest_admin_level,basemap, basemap_label, map_output):
        plt.rc('font', weight='bold')
        crs=3857
        vector_file_degrees=vector_file.copy()
        vector_file=vector_file.to_crs(crs)
        width=12
        height=8

        admin_entities= len(sub_city_gdf[admin_name])  
        if admin_entities<10:
                labelfontsize=11
                boundarylinewidth=2
                print(f"labelfontsize--->> {labelfontsize}")  
        else:
                labelfontsize=7
                boundarylinewidth=1.5
                print(f"labelfontsize--->>>> {labelfontsize}")  


        # fig = plt.figure(figsize=(width+5, height+5))
        fig = plt.figure(figsize=(width, height))
        ax = fig.add_subplot(111)
        ax = vector_file.to_crs(crs).plot(ax=ax,
                                        alpha=0.8,  
                                        facecolor='none',
                                        edgecolor='blue' , 
                                        label=f"{city}",
                                        linewidth=3,
                                        zorder=100
                                        )


        sub_city_gdf=sub_city_gdf.to_crs(crs)
        ax = sub_city_gdf.to_crs(crs).plot(ax=ax,
                                        alpha=0.8,  
                                        facecolor='none',
                                        edgecolor='cyan' ,
                                        label=f"{city}",
                                        linewidth=boundarylinewidth,
                                        missing_kwds={"color": "white", "edgecolor": "cyan", "label": "none"},
                                        zorder=5 

                                        )
        # 
        # gdf=gdf.rename(columns = {'name:en':'name_en'}, inplace = True)
        sub_city_gdf['name_en'] =sub_city_gdf[admin_name]               
        sub_city_gdf= sub_city_gdf.replace('nan', np.nan) #.dropna(how='all', axis=1, inplace=True)
        label = sub_city_gdf.dropna(subset='name_en')
        texts = []
        for i, row in label.iterrows(): 
                # print(f"Subcity name: {str(row['name_en'])}")
                texts.append(ax.annotate(text=str(row['name_en']), 
                                         xy=row.geometry.centroid.coords[0], color="black", 
                                         ha='center', fontsize=labelfontsize,  path_effects=[pe.withStroke(linewidth=1, 
                                                                                               foreground="white")]))
                
        # adjust_text(texts)
        vector_file_buffer = vector_file.to_crs(crs).buffer(3000) #for zoom out
        minx, miny, maxx, maxy = vector_file_buffer.total_bounds
        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)
    
        #Identifying how many tiles
        latlon_outline = vector_file.to_crs(WGS84).total_bounds
        def_zoom = cx.tile._calculate_zoom(*latlon_outline)
        howmany= cx.howmany(*latlon_outline, def_zoom+2, ll=True)
        print(f'Default Zoom level {def_zoom} and +2 is {howmany}')

        try:
                cx.add_basemap(ax,
                        zoom="auto",
                        # zoom=abs(def_zoom-1),
                        # zoom=def_zoom,
                        crs=vector_file.crs.to_string(),
                        source= basemap,
                        # source=cx.providers.CartoDB.PositronOnlyLabels,
                        # source=cx.providers.CartoDB.DarkMatter,
                        attribution=False, #No citation
                        # zorder = 2
                        )
        except:
                pass

        try:
                cx.add_basemap(ax,
                        # zoom="auto",
                        zoom=abs(def_zoom),
                        crs=vector_file.crs.to_string(),
                        # source=cx.providers.OpenStreetMap.Mapnik,
                        source=basemap_label, #cx.providers.CartoDB.PositronOnlyLabels,
                        # source=cx.providers.CartoDB.DarkMatter,
                        attribution=False, #No citation
                        # zorder = 3
                        )
        except:
                pass


        # try:
        #         cx.add_basemap(ax,
        #                 # zoom="auto",
        #                 zoom=abs(def_zoom-1),
        #                 crs=vector_file.crs.to_string(),
        #                 # source=cx.providers.OpenStreetMap.Mapnik,
        #                 source=basemap_label, #cx.providers.CartoDB.PositronOnlyLabels,
        #                 # source=cx.providers.CartoDB.DarkMatter,
        #                 attribution=False, #No citation
        #                 # zorder = 2
        #                 )
        # except:
        #         pass

        ax2 = inset_axes(ax, "40%", "39%", loc="upper right", bbox_to_anchor=(.078,0.01, 1,1), bbox_transform=ax.transAxes
                     ) # [left, bottom, width, height], or a tuple of [left, bottom].
        ax2 = vector_file.to_crs(crs).plot(ax=ax2, 
                                        #    alpha=0.6,
                                        #    column= visualize_column, 
                                        facecolor='none',
                                        edgecolor='blue' ,
                                        label=f"{city}", 
                                        linewidth=2,
                                        #    zorder = 2

                                        )
        ax2 = gdf_country.to_crs(crs).plot(ax=ax2, 
                                        #  alpha=0.6,
                                        facecolor='none',
                                        edgecolor='black' , 
                                        label=f"{country}",
                                        linewidth=2,
                                                # zorder = 3

                                        )
        try:
                cx.add_basemap(ax2,
                        zoom="auto",
                        # zoom=10,
                        crs=vector_file.crs.to_string(),
                        source=basemap, #cx.providers.OpenStreetMap.Mapnik,
                        attribution=False, #No citation
                        # zorder = 4
                        )
        except:
                pass

        try:
                cx.add_basemap(ax2,
                        # zoom="auto",
                        zoom=abs(def_zoom-6),
                        crs=vector_file.crs.to_string(),
                        # source=cx.providers.OpenStreetMap.Mapnik,
                        source=basemap_label, #cx.providers.CartoDB.PositronOnlyLabels,
                        # source=cx.providers.CartoDB.DarkMatter,
                        attribution=False, #No citation
                        # zorder = 4
                        )
        except:
                pass


        box, c1, c2 = mark_inset(ax2, ax, loc1=2, loc2=4, fc="none", lw=2, ec='r')
        box.set_linewidth(2)
        plt.setp([c1,c2], linewidth=0)


        LegendElement = [
                        Line2D([0],[0],
                               color='black', 
                               label=f'{country}', 
                               fillstyle=None,
                               linestyle="none", 
                               markerfacecolor=None,
                               marker="s", 
                               markersize=5.5),
                        Line2D([0],[0], 
                               color='blue', 
                               label=f'{city}',
                               linestyle="none", 
                               markerfacecolor=None,
                               fillstyle=None, 
                               marker="s", 
                               markersize=5.5),
                      Line2D([0],[0], 
                               color='cyan', 
                               label=f'{lowest_admin_level}',
                               linestyle='none', 
                               markerfacecolor=None,
                               fillstyle=None, 
                               marker="s", 
                               markersize=5.5)
                               ]
        ax2.legend(handles=LegendElement,loc='upper right', prop = { "size": 8.5, 'weight':'bold'},
                     bbox_transform=ax2.transAxes) #bbox_to_anchor=(.065,0.017, 1,1),
        # get axis position and instert the N arrow 
        # x, y , x+dx, y+dy

        gdf_country_buffer = gdf_country.to_crs(crs).buffer(20000) #for zoom out
        minx, miny, maxx, maxy = gdf_country_buffer.total_bounds
        ax2.set_xlim(minx, maxx)
        ax2.set_ylim(miny, maxy)


        ax2.axis('off')
        index=0
        polygon = vector_file_degrees.reset_index().iloc[index]['geometry']
        tags={'amenity':['college', 'dancing_school', 'driving_school', 'kindergarten', \
                        'language_school', 'library', 'surf_school', 'toy_library', 'research_institute', \
                        'training', 'music_school', 'school', 'university'],
                        'landuse': 'education'
                        }
        education=ox.features.features_from_polygon(polygon, tags)
        # tags={'leisure':['park', 'nature_reserve', 'protected_area', 'garden']}
        tags={'leisure':['park',  'garden']}
        park=ox.features.features_from_polygon(polygon, tags)
        park.geometry = park['geometry'].centroid
        tags={'landuse':['commercial', 'industrial', 'construction', 'retail']                   }
        industry=ox.features.features_from_polygon(polygon, tags)
        industry.geometry = industry['geometry'].centroid
        tags={'building': 'government'}
        government_building=ox.features.features_from_polygon(polygon, tags)
        government_building.geometry = government_building['geometry'].centroid
        tags = {"building": True}
        building_footprint=ox.features.features_from_polygon(polygon, tags)
        building_footprint.geometry = building_footprint['geometry'].centroid
        tags = {'amenity':['clinic', 'doctors', 'dentist', 'health_post', 'hospital', 'nursing_home', 'pharmacy', 'veterinary']}
        health_care=ox.features.features_from_polygon(polygon, tags)
        health_care.geometry = health_care['geometry'].centroid
        G = ox.graph.graph_from_polygon(polygon, network_type='drive')
        # add travel time based on maximum speed
        G = ox.add_edge_speeds(G) 
        G = ox.add_edge_travel_times(G) 
        G = ox.projection.project_graph(G, to_crs=3857)
        # get edges as Geo data frame
        _, gdf_edges = ox.graph_to_gdfs(G)

        ax = education.to_crs(crs).plot(ax=ax,
                                        alpha=0.6,  
                                        facecolor='yellow',
                                        edgecolor='yellow' , 
                                        label="Education",
                                        linewidth=3,
                                        markersize=10,
                                        marker="P"  
                                        )
        
        ax = industry.to_crs(crs).plot(ax=ax,
                                        alpha=0.6,  
                                        facecolor='red', 
                                        edgecolor='red' , 
                                        label="Industries",
                                        linewidth=3,
                                        markersize=10,
                                        marker="p"  
                                        )
        
        ax = park.to_crs(crs).plot(ax=ax,
                                        alpha=0.6,  
                                        facecolor='green',
                                        edgecolor='green' , 
                                        label="Parks",
                                        linewidth=3,
                                        markersize=10,
                                        marker="*"   
                                        )
        ax = health_care.to_crs(crs).plot(ax=ax,
                                        alpha=0.6,  
                                        facecolor='magenta',
                                        edgecolor='magenta' , 
                                        label="Healthcare",
                                        linewidth=3,
                                        markersize=10,
                                        marker="o"   
                                        )

        ax = gdf_edges.to_crs(crs).plot(ax=ax,
                                        alpha=0.4,  
                                        facecolor='none',
                                        edgecolor='black' , 
                                        label="Roads",
                                        linewidth=.3,
                                        linestyle='dashed',
                                        # zorder=1
                                        )


        lines = [Line2D([0], [0], linestyle="none",  fillstyle=None, marker="P", markersize=10, color='yellow' , label='Education'),
                 Line2D([0], [0], linestyle="none",  fillstyle=None, marker="p", markersize=10, color='red' , label='Industries'),
                 Line2D([0], [0], linestyle="none",  fillstyle=None, marker="*", markersize=10, color='green' , label='Parks'),
                 Line2D([0], [0], linestyle="none",  fillstyle=None, marker="o", markersize=10, color='magenta' , label='Healthcare'),
                 Line2D([0], [0], linestyle="dashed",  fillstyle=None, marker="_", markersize=10, color='black' , label='Roads')]
        ax.legend(handles=lines,loc='lower right', prop = { "size": 8 },)

        size_in_meters=5000
        asb = AnchoredSizeBar(ax.transData,
                                size=size_in_meters,
                                label=f"{int(size_in_meters/1000)} KM",
                                loc='lower left',
                                pad=0.7, borderpad=0.9, sep=5,
                                frameon=False,
                                size_vertical=size_in_meters*.03
                                # bbox_to_anchor=(0,0.001, .1,.1)
                                )
        ax.add_artist(asb)

        miny, maxy = ax.get_ylim()
        minx, maxx = ax.get_xlim()
        bounds=[minx, miny, maxx, maxy]
        print(f"City bounds:{bounds}")
        dx=maxx-minx
        dy=maxy-miny
        # N_x= minx+abs(dx/14.95) 
        N_x= minx+abs(dx/17) 
        N_y= miny+abs(dy/7)
        arrow_sign_x=minx+abs(dx/15)
        arrow_sign_y=miny+abs(dy/11)

        head_width=abs(dy/28)
        overhang=.1
        head_length=1.5*head_width
        ax.text(x=N_x, y=N_y, s='N', fontsize=15)
        ax.arrow(arrow_sign_x, arrow_sign_y, dx=dx*0, dy=head_width , length_includes_head=True,
                head_width=head_width, head_length=head_length, overhang=overhang, facecolor='k')

        ax.axis('off')
        # ax.set_title(f'{city}', fontdict={'fontsize': '16', 'fontweight' : '6'})
        # ax.annotate('Source: OSM basemaps',xy=(0.1, .1),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=6, color='#555555')
        plt.xticks(visible=False)
        plt.yticks(visible=False)
        plt.tight_layout()
        ax.figure.savefig(map_output,  bbox_inches='tight',   dpi = 400, pad_inches=0)
        return education, park, industry, government_building, building_footprint, gdf_edges


: 


Section In Progress: generate text etc
Union councils or upazils subdistricts level analysis. Hum + OSM. If no subcity then go for clustering


In [ ]:
def get_profiles(polygonized_shapefile,shp,WGS84,subdir,visualize_column, title, legend_title,cmap):
     # adjust_text(texts)
     vector_file = gpd.read_file(shp).to_crs(WGS84)
     vector_file_buffer = vector_file.to_crs(WGS84).buffer(-0.02) #-2km buffer
     # vector_file_buffer = vector_file.to_crs(WGS84)
     minx, miny, maxx, maxy = vector_file_buffer.total_bounds
     minx, miny, maxx, maxy
     midy= miny+ (maxy-miny)/2
     midx= minx+ (maxx-minx)/2
     minx, miny, maxx, maxy, midx,midy
     left_start=Point(minx, midy)
     right_end=Point(maxx, midy)
     top_start=Point(midx, maxy)
     bottom_end=Point(midx, miny)

     length = 0.013
     angle = -math.pi/8
     line = LineString([left_start, right_end])
     # for restircting line within the boundary
     gdf_line = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[line])
     line_spat_join_city_horizontal=gdf_line
     # line_spat_join_city_horizontal =overlay(gdf_line.to_crs(crs), vector_file.to_crs(crs), how="intersection")
     # line_spat_join_city_horizontal = gpd.sjoin(vector_file.to_crs(crs), gdf_line.to_crs(crs), how="right", predicate='intersects')

     line = LineString([top_start, bottom_end])
     # for restircting line within the boundary
     gdf_line = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[line])
     # line_spat_join_city_vertical =overlay(gdf_line.to_crs(crs), vector_file.to_crs(crs), how="intersection")
     # line_spat_join_city_vertical =overlay(gdf_line.to_crs(crs), vector_file.to_crs(crs), how="identity")
     line_spat_join_city_vertical = gpd.sjoin(vector_file.to_crs(crs), gdf_line.to_crs(crs), how="right", predicate='contains')
     fig = plt.figure(figsize=(width, height))
     ax = fig.add_subplot(111)
     ax = polygonized_shapefile.to_crs(crs).plot(ax=ax, column= visualize_column, 
                                   linewidth=0 , alpha=0.6, scheme="quantiles", cmap=cmap, 
                                   legend=True,  # Add legend 
                                   legend_kwds={'loc':'upper right', 
                                                'bbox_to_anchor':(1.01, 1.01), 
                                                'fmt':legends_format,
                                                'markerscale':1.01, 
                                                'title_fontsize':'small', 
                                                'fontsize':'small',
                                                # 'fontweight': 'bold'
                                                }  
                                        )
     vector_file.to_crs(crs).plot(ax=ax,facecolor='none',edgecolor='k',alpha=0.2)
     cx.add_basemap(ax, 
                   source=cx.providers.Esri.WorldImagery,  
                   crs=crs,
                   attribution=False, #No citation
                   ) 
     # plt.title(f'{title}',fontsize=16)
     plt.xticks([])
     plt.yticks([])
          
     leg1 = ax.get_legend()
     # Set markers to square shape
     for ea in leg1.legendHandles:
          ea.set_marker('s')
     leg1.set_title(f'{legend_title}')
     # ax.set_title(title, fontsize=12,  fontweight='bold', wrap=True)
     ax.set_title(title,  fontweight='bold', wrap=True)
     # plt.subplots_adjust(top = 0.00001, bottom = 0.00001, right = 0, left = 0, hspace = 0, wspace = 0)
     

     ax = line_spat_join_city_horizontal.to_crs(crs).plot(ax=ax,
                                   alpha=0.6,  
                                   facecolor='yellow',
                                   edgecolor='yellow' , 
                                   label="Horizontal",
                                   linewidth=3,
                                   markersize=10,
                                   marker="o"   
                                   )

     ax = line_spat_join_city_vertical.to_crs(crs).plot(ax=ax,
                                   alpha=0.6,  
                                   facecolor='cyan',
                                   edgecolor='cyan' , 
                                   label="Top down",
                                   linewidth=3,
                                   markersize=10,
                                   marker="o"   
                                   )

     
     #  horizontal line profile
     lines_read= line_spat_join_city_horizontal #gpd.read_file(f"{shapefiles}/Poly_1.shp").to_crs(crs)
     # newdf = gpd.sjoin(hexagons.to_crs(crs), lines_read)
     for ind, row in lines_read.iterrows():
          # XS_ID = row['id']
          start_coords =  list([row.geometry][0].coords)[0]
          end_coords = list([row.geometry][0].coords)[1]
          lon = [start_coords[0]]
          lat = [start_coords[1]]
          n_points = 10
          for i in np.arange(1, n_points+1):
               x_dist = end_coords[0] - start_coords[0]
               y_dist = end_coords[1] - start_coords[1]
               point  = [(start_coords[0] + (x_dist/(n_points+1))*i), (start_coords[1] + (y_dist/(n_points+1))*i)]
               lon.append(point[0])
               lat.append(point[1])
               
          lon.append(end_coords[0])
          lat.append(end_coords[1])
          df = pd.DataFrame({'Latitude': lat, 
                              'Longitude': lon})
          gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))
          gdf.crs = {'init': 'epsg:4326'}
          gdf_pcs = gdf.to_crs(epsg = 3857)
          print(gdf_pcs.head())
          gdf_pcs['h_distance'] = 0
          for index, row in gdf_pcs.iterrows():
               gdf_pcs['h_distance'].loc[index] = gdf_pcs.geometry[0].distance(gdf_pcs.geometry[index])

          profile_x_var='h_distance'
          visualize_column= subdir[0:9]  

          for x, y in zip(lon[::1], lat[::1]):
               value="----------"
               ax.text(x, y, f"{value}", ha="center", va="center", rotation=angle, size=1.5,
                         bbox=dict(boxstyle="rarrow,pad=0.4", fc="yellow", ec="yellow", lw=2))


     fig, axh = plt.subplots(figsize = (4.5, 3.5))
     line_spat_join = gpd.sjoin(polygonized_shapefile.to_crs(crs), gdf_pcs.to_crs(crs), how="right", predicate='contains')
     axh.plot(line_spat_join[profile_x_var], line_spat_join[f'{visualize_column}'])
     axh.set_title(label=f"{legend_title} Profile (Horizonal)" , fontdict={'fontsize': 11, 'fontweight': 'bold'})
     axh.set_xlabel("Distance (KM)",  fontdict={'fontsize': 11, 'fontweight': 'bold'})
     axh.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.1f}'.format(x/1000)))
     axh.set_ylabel(f"{legend_title}", fontdict={'fontsize': 11, 'fontweight': 'bold'})

     #----------------------
     max_value= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()][f'{visualize_column}'].round()
     Latitude= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()]['Latitude']
     Longitude= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()]['Longitude']
     max_distance= line_spat_join.loc[(line_spat_join['Latitude'] == Latitude) & (line_spat_join['Longitude'] == Longitude)]['h_distance']
     thisr, thistheta =  max_value, max_distance
     axh.plot([thistheta], [thisr], 'o')
     axh.annotate(
     f'Max:{int(max_value)}',
     xy=(thistheta, thisr), xycoords='data',
     xytext=(30, -2), textcoords='offset points',
     arrowprops=dict(arrowstyle="->")
     )

     plt.xticks(fontsize=11) # rotation=90
     plt.yticks(fontsize=11) 
     # Hide the right and top spines
     axh.spines[['right', 'top']].set_visible(False)
     plt.tight_layout()
     axh.figure.savefig(f"{maps}/{country}_{city}_{subdir}_profile_horizontal_{visualize_column}.png",  bbox_inches='tight', dpi=60) #, transparent=True  
     line_spat_join.to_csv(f"{tables}/{country}_{city}_{subdir}_profile_horizontal_{visualize_column}.csv")


     #  vertical line profile
     # angle = 0
     angle = -90
     lines_read= line_spat_join_city_vertical #gpd.read_file(f"{shapefiles}/Poly_1.shp").to_crs(crs)
     # newdf = gpd.sjoin(hexagons.to_crs(crs), lines_read)
     for ind, row in lines_read.iterrows():
          # XS_ID = row['id']
          start_coords =  list([row.geometry][0].coords)[0]
          end_coords = list([row.geometry][0].coords)[1]
          lon = [start_coords[0]]
          lat = [start_coords[1]]
          n_points = 10
          for i in np.arange(1, n_points+1):
               x_dist = end_coords[0] - start_coords[0]
               y_dist = end_coords[1] - start_coords[1]
               point  = [(start_coords[0] + (x_dist/(n_points+1))*i), (start_coords[1] + (y_dist/(n_points+1))*i)]
               lon.append(point[0])
               lat.append(point[1])
               
     lon.append(end_coords[0])
     lat.append(end_coords[1])
     df = pd.DataFrame({'Latitude': lat, 
                         'Longitude': lon})
     gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))
     gdf.crs = {'init': 'epsg:4326'}
     gdf_pcs = gdf.to_crs(epsg = 3857)
     print(gdf_pcs.head())
     gdf_pcs['h_distance'] = 0
     for index, row in gdf_pcs.iterrows():
          gdf_pcs['h_distance'].loc[index] = gdf_pcs.geometry[0].distance(gdf_pcs.geometry[index])

     profile_x_var='h_distance'
     visualize_column= subdir[0:9]  
     for x, y in zip(lon[::1], lat[::1]):
          value="----------"
          ax.text(x, y, f"{value}", ha="center", va="center", rotation=angle, size=1.5,
                    bbox=dict(boxstyle="rarrow,pad=0.4", fc="cyan", ec="cyan", lw=2))

     fig, axv = plt.subplots(figsize = (4.5, 3.5))
     line_spat_join = gpd.sjoin(polygonized_shapefile.to_crs(crs), gdf_pcs.to_crs(crs), how="right", predicate='contains')
     axv.plot(line_spat_join[profile_x_var], line_spat_join[f'{visualize_column}'])
     axv.set_title(label=f"{legend_title} Profile (Vertical)" , fontdict={'fontsize': 11, 'fontweight': 'bold'})
     axv.set_xlabel("Distance (KM)",  fontdict={'fontsize': 11, 'fontweight': 'bold'})
     axv.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.1f}'.format(x/1000)))
     axv.set_ylabel(f"{legend_title}", fontdict={'fontsize': 11, 'fontweight': 'bold'})

     #----------------------
     max_value= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()][f'{visualize_column}'].round()
     Latitude= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()]['Latitude']
     Longitude= line_spat_join.loc[line_spat_join[f'{visualize_column}'].idxmax()]['Longitude']
     max_distance= line_spat_join.loc[(line_spat_join['Latitude'] == Latitude) & (line_spat_join['Longitude'] == Longitude)]['h_distance']
     thisr, thistheta =  max_value, max_distance
     axv.plot([thistheta], [thisr], 'o')
     axv.annotate(
     f'Max:{int(max_value)}',
     xy=(thistheta, thisr), xycoords='data',
     xytext=(30, -2), textcoords='offset points',
     arrowprops=dict(arrowstyle="->")
     )

     plt.xticks(fontsize=11) # rotation=90
     plt.yticks(fontsize=11) 
     # Hide the right and top spines
     axv.spines[['right', 'top']].set_visible(False)
     plt.tight_layout()
     axv.figure.savefig(f"{maps}/{country}_{city}_{subdir}_profile_vertical_{visualize_column}.png",  bbox_inches='tight', dpi=60) #, transparent=True  
     line_spat_join.to_csv(f"{tables}/{country}_{city}_{subdir}_profile_vertical_{visualize_column}.csv")

     ax.axis('off')
     plt.tight_layout()
     ax.figure.savefig(f"{maps}/{country}_{city}_{subdir}_profiles_{visualize_column}.png",  bbox_inches='tight', dpi=400) #, transparent=True  
     
def get_services_infrastructure_inside_poly(polygon):
     try:
          tags={'leisure':['park', 'nature_reserve', 'protected_area', 'garden']}
          park=ox.features.features_from_polygon(polygon, tags)
     except:
          park=0
          pass
     try:
          tags={'amenity':['college', 'dancing_school', 'driving_school', 'kindergarten', \
                         'language_school', 'library', 'surf_school', 'toy_library', 'research_institute', \
                              'training', 'music_school', 'school', 'university'],
                              'landuse': 'education'
                              }
          education=ox.features.features_from_polygon(polygon, tags)
     except:
          education=0
          pass
     try:
          tags={'landuse':['commercial', 'industrial', 'construction', 'retail']                   }
          industry=ox.features.features_from_polygon(polygon, tags)
     except:
          industry=0
          pass
     try:
          tags={'building': 'government'}
          building=ox.features.features_from_polygon(polygon, tags)
     except:
          building=0
          pass
     # print("park , education, industry, building:", park , education, industry, building)
     return park , education, industry, building

def create_ahc_knn_clusters(db  , raster_val , WGS84_meters):
    raster_val=raster_val[0:9]
    db =db.to_crs(WGS84_meters)
    db['area_sqm'] = db.geometry.area
    cluster_variables = [raster_val]
    db_scaled = robust_scale(db[cluster_variables])
    w = KNN.from_dataframe(db, k=3) #k for four nearest neighbors
    # Specify cluster model with spatial constraint
    model = AgglomerativeClustering(
        linkage="ward", connectivity=w.sparse, n_clusters=5
    )
    # Fit algorithm to the data
    model.fit(db_scaled)
    db["ward5wknn"] = model.labels_
    db['ward5wknn_sum'] = db[cluster_variables].groupby(db['ward5wknn']).transform('sum')
    db['area_sum'] = db['area_sqm'].groupby(db['ward5wknn']).transform('sum')
    db['area_sum'] = db['area_sum'].apply(lambda x: x/10000) #Hectares
    visualize_var= 'ward5wknn_sum_per_meter'
    db[visualize_var] = db['ward5wknn_sum'].div(db['area_sum']).round(2)
    # select the columns that you with to use for the dissolve and that will be retained
    cluster_boundaries = db[[raster_val , visualize_var,"ward5wknn", "geometry", "area_sqm", "area_sum", "ward5wknn_sum"]]
    # dissolve the state boundary by region 
    cluster_boundaries = cluster_boundaries.dissolve(by="ward5wknn" , aggfunc={raster_val: "sum", visualize_var: "sum" , "area_sqm":"sum" , "area_sum": "sum", "ward5wknn_sum": "sum",} , )
    cluster_boundaries=cluster_boundaries.explode().reset_index() #convert multipolygons to single part polygons. Multis cause errors
    cluster_boundaries["area_for_dupes"]=cluster_boundaries.geometry.area
#     cluster_boundaries=cluster_boundaries.sort_values('area_for_dupes', ascending=False).drop_duplicates(["ward5wknn"])
    cluster_boundaries=multi2single(cluster_boundaries)
    cluster_boundaries = cluster_boundaries.to_crs(crs)
    cluster_boundaries["centroid"] = cluster_boundaries["geometry"].centroid
    cluster_boundaries['lon'] = cluster_boundaries['centroid'].x
    cluster_boundaries['lat'] = cluster_boundaries['centroid'].y
    cluster_boundaries['cluster_area_deg'] = cluster_boundaries.geometry.area
    return db , cluster_boundaries, visualize_var



def describe_cluster_trees_pop(cluster_boundaries, category, maps):
     category=category[0:9]
     df = pd.DataFrame(cluster_boundaries.drop(columns='geometry'))
     df = df.sort_values('cluster_area_deg', ascending = False).groupby('ward5wknn').head(2)
     df = df.reset_index() 
     df['Rank'] = df['ward5wknn_sum_per_meter'].rank(method='dense', ascending=False)
     df= df.sort_values(['Rank'], ascending=[True])
     cent= cluster_boundaries.dissolve().centroid 
     # White house 38.8977° N, 77.0365° W. centroid of the city.
     lat1 = float(cent.centroid.y)
     long1= float(cent.centroid.x)
     list_statements= []
     stats_dict= defaultdict(list)
     for index, row in df.iterrows():
          lat2 = row['lat'] # cent.centroid.y  #38.8893
          long2 =row['lon'] # cent.centroid.x  # -77.0506
          # print(f"lat1 {lat1} , long1 {long1}, lat2 {lat2}, long2 {long2}")
          points = calcNSEW(lat1, long1, lat2, long2)
          rank= int(row['Rank'])
          print(f"rank------>>> {rank}")
          # rank= ordinal(rank)
          value= row['ward5wknn_sum_per_meter']
          statement= f"A cluster that is located in the {points} of the city, is ranked {rank} and has the value of {value}"
          list_statements.append(statement)
          print(f"statement:::: {statement}")
          try:
               polygon = cluster_boundaries.iloc[index]['geometry']
               cluster_area_deg= cluster_boundaries.iloc[index]['cluster_area_deg']
               ward5wknn= cluster_boundaries.iloc[index]['ward5wknn']
               park , education, industry, building=get_services_infrastructure_inside_poly(polygon)
               stats_dict["cluster"].append(index)
               stats_dict["ward5wknn"].append(ward5wknn)
               stats_dict["points"].append(points)
               stats_dict["cluster_area_deg"].append(cluster_area_deg)
               stats_dict["rank"].append(rank)
               stats_dict["value"].append(value)
          except:
               pass

          try:
               stats_dict["park"].append(len(park))
          except:
               pass

          try:
               stats_dict["education"].append(len(education))
          except:
               pass

          try:
               stats_dict["industry"].append(len(industry))
          except:
               pass

          try:
               stats_dict["building"].append(len(building))
          except:
               pass

          try:
               
               park["ward5wknn"]=ward5wknn
               park["index"]=index
               park["points"]=points
               park["rank"]=rank
               park["value"]=value
               park.to_csv(f"{tables}/{ward5wknn}_{category}_park_df.csv")
          except:
               pass

          try:
               education["ward5wknn"]=ward5wknn
               education["index"]=index
               education["points"]=points
               education["rank"]=rank
               education["value"]=value
               education.to_csv(f"{tables}/{ward5wknn}_{category}_education_df.csv")
          except:
               pass

          try:
               industry["ward5wknn"]=ward5wknn
               industry["index"]=index
               industry["points"]=points
               industry["rank"]=rank
               industry["value"]=value
               industry.to_csv(f"{tables}/{ward5wknn}_{category}_industry_df.csv")
          except:
               pass

          try:
               building["ward5wknn"]=ward5wknn
               building["index"]=index
               building["points"]=points
               building["rank"]=rank
               building["value"]=value
               building.to_csv(f"{tables}/{ward5wknn}_{category}_building_df.csv")               
          except:
               pass

          try:
               stats=get_network_stats(polygon)
               stats = stats.to_frame(0).T
               # category=category[0:9]
               stats.to_csv(f"{tables}/{ward5wknn}_{category}_infra_network_stats.csv")
          except:
               print(f"no network for {ward5wknn}")
               pass
     
     try:
          statement_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in stats_dict.items() ]))
          # category=category[0:9]
          statement_df.to_csv(f"{tables}/{category}_amenities_stats_all.csv")
     except:
          print("No amenities")
          pass

     return list_statements


def describe_db( db_var, category, tables):
     try:
          category=category[0:9]
          cluster_variables= category
          k5desc = db_var.groupby('ward5wknn')[cluster_variables].describe()
          df1=k5desc.reset_index(0).reset_index(drop=True)
          # category=category[0:9]
          df1.to_csv(f"{tables}/{category}_summary_stats_df.csv")
     except:
          pass

def multi2single(gpdf):
    gpdf_singlepoly = gpdf[gpdf.geometry.type == 'Polygon']
    gpdf_multipoly = gpdf[gpdf.geometry.type == 'MultiPolygon']

    for i, row in gpdf_multipoly.iterrows():
        Series_geometries = pd.Series(row.geometry)
        df = pd.concat([gp.GeoDataFrame(row, crs=gpdf_multipoly.crs).T]*len(Series_geometries), ignore_index=True)
        df['geometry']  = Series_geometries
        gpdf_singlepoly = pd.concat([gpdf_singlepoly, df])

    gpdf_singlepoly.reset_index(inplace=True, drop=True)
    return gpdf_singlepoly


def calcBearing (lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    bearing = math.atan2(x,y)   # use atan2 to determine the quadrant
    bearing = math.degrees(bearing)
    return bearing

def calcNSEW(lat1, long1, lat2, long2):
    points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    bearing = int(bearing / 45) # values 0 to 7
    NSEW = points [bearing]
    return NSEW

def ordinal(n: int):
    if 11 <= (n % 100) <= 13:
        suffix = 'th'
    else:
        suffix = ['th', 'st', 'nd', 'rd', 'th'][min(n % 10, 4)]
    return str(n) + suffix


def get_network_stats(polygon):
     try:
          # polygon = cluster_boundaries.iloc[index]['geometry'] 
          graph = ox.graph.graph_from_polygon(polygon, network_type='drive')
          # Retrieve only edges from the graph
          edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
          graph_proj = ox.project_graph(graph)
          # fig, ax = ox.plot_graph(graph_proj) 
          # Get Edges and Nodes
          nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)
          # print("Coordinate system:", edges_proj.crs)
          # edges_proj.head()
          # Get Edges and Nodes
          nodes_proj, edges_proj = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)
          # print("Network Coordinate system: projected", edges_proj.crs)
          # edges_proj.head()
          # Get the Convex Hull of the network
          convex_hull = edges_proj.unary_union.convex_hull
          # Calculate the area
          area = convex_hull.area
          # Calculate statistics with density information
          stats = ox.stats.basic_stats(graph_proj, area=area) #replace area by city polygon
          stats =pd.Series(stats) #edge_density_km(road density) - edge_length_total per sq km
     except:
          stats=0
          stats =pd.Series(stats) 
          pass     
     return stats



: 

In [ ]:
db , cluster_boundaries, visualize_var =create_ahc_knn_clusters(db = polygonized_shapefile , raster_val=raster_val, 
                                                              WGS84_meters=WGS84_meters)
category=raster_val
describe_cluster_trees= describe_cluster_trees_pop(cluster_boundaries, category, maps)
map_title= f"Population across {city}"
legends_format= '{:,.0f}'
categorical=True
scheme="quantiles"

map_output= f'{maps}/{city}_{subdir}_{raster_val}_2.png'
export_the_map= export_map(map_dir=maps, vector_file=vector_file , \
        hexagons=db,legend_title=legend_title, legends_format=legends_format,  \
        crs=crs, cmap=cmap , visualize_column=raster_val,  \
        title=map_title, map_output=map_output)

cluster_boundaries[visualize_var]=cluster_boundaries[visualize_var].round(0)

map_title= f"Population across clusters(Population/Hectares)"
legends_format= '{:,.f}'
categorical=True
map_output= f'{maps}/{city}_{subdir}_{raster_val}_{visualize_var}_map_clusters.png'
export_map_categorical(vector_file=vector_file , hexagons=cluster_boundaries,legend_title=legend_title, 
                       legends_format=legends_format, crs=WGS84 
                       ,cmap=cmap,  visualize_column=visualize_var, title=map_title, map_output=map_output)

visualize_column="ward5wknn_sum"
map_title= f"Population across clusters(Total Population)"

map_output= f'{maps}/{city}_{subdir}_{raster_val}_{visualize_column}_map_clusters.png'
export_map_categorical(vector_file=vector_file , hexagons=cluster_boundaries,legend_title=legend_title, 
                       legends_format=legends_format, crs=WGS84 
                       ,cmap=cmap,  visualize_column=visualize_column, title=map_title, map_output=map_output)


: 

In [ ]:
from docx import Document
from docx.shared import Inches

document = Document()
p = document.add_paragraph()
r = p.add_run()
r.add_text('Good Morning every body,This is my ')
r.add_picture(map_output)
r.add_text(' do you like it?')

document_output = f"{tables}/{country}_{city}_fcs_report.docx"
document.save(document_output)

: 

In [ ]:
                        
def export_figure_with_heading( document_output, mappath, title):  
    document = Document(document_output)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text(title)
    r.add_picture(mappath)
    r.add_text('Notes here')
    document.save(document_output)
    # return print('doc saved')


# document_output = f"{tables}/{country}_{city}_Future_city_scan_document.docx"
# export_dox= export_figure_with_heading( document_output = document_output, mappath=map_output, title=map_title)


: 

Create modules and package then import them

In [ ]:
from section_1_population import export_popdynamics

: 

In [ ]:
# %%time
if __name__ ==  '__main__':    
    from utils import load_shapefile
    load_shapefile(shp)


: 

In [ ]:
I want all three ax objects to share the same bins (preferable the central_pop scenario quantiles) so that the legend is consistent for the whole figure.
This way I should see darker colors (more red) in the far right ax showing the high_pop scenario.

How can I set the colorscheme bins for the whole figure / each of the ax objects?

The simplest way I can see this working is either
a) Provide a set of bins to the geopandas.plot() function
b) extract the colorscheme / bins from one ax and apply it to another.

@knaaptime
knaaptime commented on Jun 21, 2019 • 
Under the hood, geopandas uses mapclassify, and the easiest way to achieve what you want would be to just use it directly:

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from mapclassify import Quantiles, User_Defined

# Note you can read directly from the URL
gdf = gpd.read_file('https://opendata.arcgis.com/datasets/8edafbe3276d4b56aec60991cbddda50_1.zip?outSR=%7B%22latestWkid%22%3A27700%2C%22wkid%22%3A27700%7D&session=850489311.1553456889'
)

# 380 values
df = pd.DataFrame([])
df['AREA_CODE'] = gdf.lad15cd.values
df['central_pop'] = np.random.normal(30, 15, size=(len(gdf.lad15cd.values)))
df['low_pop'] = np.random.normal(10, 15, size=(len(gdf.lad15cd.values)))
df['high_pop'] = np.random.normal(50, 15, size=(len(gdf.lad15cd.values)))

def join_df_to_shp(pd_df, gpd_gdf):
    """"""
    df_ = pd.merge(pd_df, gpd_gdf[['lad15cd','geometry']], left_on='AREA_CODE', right_on='lad15cd', how='left')

    # DROP the NI counties
    df_ = df_.dropna(subset=['geometry'])

    # convert back to a geopandas object (for ease of plotting etc.)
    crs = {'init': 'epsg:4326'}
    gdf_ = gpd.GeoDataFrame(df_, crs=crs, geometry='geometry')
    # remove the extra area_code column joined from gdf
    gdf_.drop('lad15cd',axis=1, inplace=True)

    return gdf_

pop_gdf = join_df_to_shp(df, gdf)

fig,(ax1,ax2,ax3,) = plt.subplots(1,3,figsize=(15,6))

# define your bins
bins = Quantiles(pop_gdf['central_pop'], 5).bins

# create a new column with the discretized values and plot that col
# repeat for each view
pop_gdf.assign(cl=User_Defined(df['low_pop'].dropna(), bins).yb).plot(
    column='cl', ax=ax1, cmap='OrRd'
)
pop_gdf.assign(cl=User_Defined(df['central_pop'].dropna(), bins).yb).plot(
    column='cl', ax=ax2, cmap='OrRd',
)
pop_gdf.assign(cl=User_Defined(df['high_pop'].dropna(), list(bins)).yb).plot(
    column='cl', ax=ax3, cmap='OrRd',
)
for ax in (ax1,ax2,ax3,):
    ax.axis('off')

: 

In [ ]:
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.xkcd()
seoul = gpd.read_file('./data/seoul_municipalities_geo.json')
# You can download the same file from the Github below
# https://github.com/southkorea/seoul-maps/tree/master/juso/2015/json

seoul.plot(figsize=(14,10),linewidth=0.25, edgecolor='black', column='ESRI_PK', cmap='Blues', scheme='quantiles')

for index, row in seoul.iterrows():
    xy = row['geometry'].centroid.coords[:]
    xytext = row['geometry'].centroid.coords[:]
    plt.annotate(row['SIG_ENG_NM'], xy=xy[0], xytext=xytext[0], horizontalalignment='center', verticalalignment='center')
    plt.axis('off')

cmap = cm.get_cmap('Blues')
white_patch = mpatches.Patch(color=cmap(0.0), label='0 - 5')
lowblue_patch = mpatches.Patch(color=cmap(0.25), label='5 - 10')
midblue_patch = mpatches.Patch(color=cmap(0.5), label='10 - 14')
highblue_patch = mpatches.Patch(color=cmap(0.75), label='14 - 19')
veryblue_patch = mpatches.Patch(color=cmap(1.0), label='19 - 24')
plt.legend(handles=[white_patch, lowblue_patch, midblue_patch, highblue_patch, veryblue_patch])

plt.show()

: 

: 

: 

In [ ]:
# Stop
# Visulizie %
# Fathom vars
# set up one_drive folder for maps
# Ask Nick for heat layer
# Fathom + TextDisplayObject
# Minor cosmetic changes

: 

: 

: 